In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 31.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [4]:
import google.generativeai as genai
from IPython.display import Markdown

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


In [7]:
# import os
# from PyPDF2 import PdfReader

# def extract_text_from_pdfs(folder_path):
#     documents = []
#     for file in os.listdir(folder_path):
#         if file.endswith(".pdf"):
#             reader = PdfReader(os.path.join(folder_path, file))
#             text = ""
#             for page in reader.pages:
#                 text += page.extract_text()
#             documents.append({"content": text, "metadata": {"source": file}})
#     return documents

# # Extract text from journal PDFs
# journal_folder = "/kaggle/input/lungca-tki-resistance"
# journal_data = extract_text_from_pdfs(journal_folder)


In [8]:
import os
from PyPDF2 import PdfReader

def extract_text_from_pdfs(folder_path):
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            try:
                reader = PdfReader(os.path.join(folder_path, file))
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
                documents.append({"content": text, "metadata": {"source": file}})
            except Exception as e:
                print(f"Error processing file {file}: {e}")
    return documents

# Extract text from journal PDFs
journal_folder = "/kaggle/input/lungca-tki-resistance"
journal_data = extract_text_from_pdfs(journal_folder)
pdf_files = [file for file in os.listdir(journal_folder) if file.endswith(".pdf")]
print(f"Number of PDF files in folder: {len(pdf_files)}")

journal_data = extract_text_from_pdfs(journal_folder)
print(f"Number of PDFs successfully loaded: {len(journal_data)}")

# Verify all PDFs are loaded
if len(pdf_files) == len(journal_data):
    print("All PDFs were successfully loaded.")
else:
    print("Some PDFs could not be loaded.")


Number of PDF files in folder: 22
Number of PDFs successfully loaded: 22
All PDFs were successfully loaded.


In [9]:
documents = journal_data

In [10]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
import chromadb


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [11]:
import chromadb

DB_NAME = "lungcatki"

# Ensure that GeminiEmbeddingFunction is correctly initialized
# Replace this with your embedding function initialization
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# Validate and sanitize the documents
try:
    # Check if documents is defined and contains valid strings
    if not isinstance(documents, list):
        raise ValueError("Expected 'documents' to be a list.")
    
    # Ensure all items in documents are strings
    documents = [str(doc) for doc in documents]
    
    # Add documents to the collection
    db.add(documents=documents, ids=[str(i) for i in range(len(documents))])
except Exception as e:
    print(f"An error occurred: {e}")


In [12]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "The EGFR T790 has a mutation that causes steric hindrance, where is the attachement on the EGFR protein does this happen, what is the amino acid chain of this mutation, what can be done to clip of steric hindrance?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

{'content': 'Shi\xa0 et\xa0al. Journal of Hematology & Oncology           (2022) 15:94  \nhttps://doi.org/10.1186/s13045-022-01311-6\nREVIEW\nEmerging strategies to\xa0overcome resistance \nto\xa0third- generation EGFR inhibitors\nKunyu Shi1,5†, Guan Wang1†, Junping Pei1†, Jifa Zhang1,5, Jiaxing Wang2, Liang Ouyang1,3,5*, Yuxi Wang1,4,5* and \nWeimin Li1,4,5* \nAbstract \nEpidermal growth factor receptor (EGFR), the receptor for members of the epidermal growth factor family, regulates \ncell proliferation and signal transduction; moreover, EGFR is related to the inhibition of tumor cell proliferation, angio -\ngenesis, invasion, metastasis, and apoptosis. Therefore, EGFR has become an important target for the treatment of \ncancer, including non-small cell lung cancer, head and neck cancer, breast cancer, glioma, cervical cancer, and bladder \ncancer. First- to third-generation EGFR inhibitors have shown considerable efficacy and have significantly improved \ndisease prognosis. However, most patients develop drug resistance after treatment. The challenge of overcoming \nintrinsic and acquired resistance in primary and recurrent cancer mediated by EGFR mutations is thus driving the \nsearch for alternative strategies in the design of new therapeutic agents. In view of resistance to third-generation \ninhibitors, understanding the intricate mechanisms of resistance will offer insight for the development of more \nadvanced targeted therapies. In this review, we discuss the molecular mechanisms of resistance to third-generation \nEGFR inhibitors and review recent strategies for overcoming resistance, new challenges, and future development \ndirections.\nKeywords: Epidermal growth factor receptor (EGFR), Drug resistance, Inhibitors, Structure–activity relationship, \nTyrosine kinase, Cancer\n© The Author(s) 2022. Open Access This article is licensed under a Creative Commons Attribution 4.0 International License, which \npermits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the \noriginal author(s) and the source, provide a link to the Creative Commons licence, and indicate if changes were made. The images or \nother third party material in this article are included in the article’s Creative Commons licence, unless indicated otherwise in a credit line \nto the material. If material is not included in the article’s Creative Commons licence and your intended use is not permitted by statutory \nregulation or exceeds the permitted use, you will need to obtain permission directly from the copyright holder. To view a copy of this \nlicence, visit http:// creat iveco mmons. org/ licen ses/ by/4. 0/. The Creative Commons Public Domain Dedication waiver (http:// creat iveco  \nmmons. org/ publi cdoma in/ zero/1. 0/) applies to the data made available in this article, unless otherwise stated in a credit line to the data.Introduction\nEpidermal growth factor receptor (EGFR) is a member \nof the receptor tyrosine kinase (RTK) superfamily that \nconsists of exon boundaries and associated extracellular, \ntransmembrane, and intracellular protein domains. EGFR \nis involved in multiple signaling pathways and regulates \nnumerous cell functions (Fig.\xa0 1A). This transmembrane \nglycoprotein is composed of a cysteine-rich extracellu -\nlar ligand binding domain, hydrophobic transmembrane domain, cytoplasmic RTK domain, and C-terminal \ndomain. The RTK domain contains an N-lobe consist -\ning of five β-sheet strands and one αC helix and a C-lobe \ncontaining the main helices of a highly flexible activa -\ntion loop (A-loop) [1]. The deep cleft at the junction of \nthese two lobes forms the binding pocket for the adenine \nring of ATP . The conformation of three conserved struc -\ntural elements, namely the Asp-Phe-Gly (DFG) motif, \nαC helix, and A-loop, critically regulates the activation \nor inactivation of the catalytic domain. When EGFR is \nin the active state, the important catalytic residue D855 \nis located in the ATP binding site, stabilizing the ATP-\nloaded complex (DFG-in) and αC helix (αC-in). In the \ninactive state, EGFR forms a Src-like structure, including \na closed A-loop, αC-out, and DFG-in [2]. (Fig.\xa0 1B). EGFR \ncan dimerize upon binding by ligands, such as amphireg -\nulin, β-cytokines, epidermal growth factor (EGF), Open Access\n†Kunyu Shi, Guan Wang and Junping Pei contributed equally to this work\n*Correspondence:  ouyangliang@scu.edu.cn; yuxiwang@scu.edu.cn; \nweimi003@scu.edu.cn\n1 Department of Respiratory and Critical Care Medicine, Targeted Tracer \nResearch and Development Laboratory, Institute of Respiratory Health, \nInnovation Center of Nursing Research, Nursing Key Laboratory of Sichuan \nProvince, West China Hospital, Sichuan University, Chengdu 610041, China\nFull list of author information is available at the end of the articlePage 2 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nheparin-binding EGF-like growth factor (HB-EGF), and \ntransforming growth factor (TGF). The activation of the \nintracellular tyrosine kinase domain and autophospho -\nrylation, which initiates the Ras/RAF/MEK, signal trans -\nducer and activator of transcription (STAT), PI3K/AKT/\nmTOR and other downstream signaling pathways, are \nclosely related to embryonic development and stem cell \ndivision [2–4]. Overexpression of wild-type (WT) EGFR \nprotein with or without EGFR gene amplification or a \nkinase-activating mutation further enhances cell prolif -\neration, migration, survival, and antiapoptotic responses \nthrough signaling cascades, and these processes are \nclosely related to the occurrence and development of many types of epithelial-derived cancer, such as non-\nsmall cell lung cancer (NSCLC), breast cancer, glioma, \nhead and neck cancer, cervical cancer, and bladder can -\ncer. Among these cancers, lung cancer appears to be the \nmost common and has the characteristics of aberrant \nproliferation, metastasis, and drug resistance [5–8]. Thus, \nEGFR has become a promising target for anticancer drug \ndesign and development. EGFR tyrosine kinase inhibi -\ntors (EGFR-TKIs) have achieved remarkable results in \nthe clinic [9]. However, most patients develop acquired \ndrug resistance to first- and second-generation EGFR-\nTKIs after 1–2\xa0years. The mechanism of drug resistance \nfor nearly half of cases relates to the T790M mutation. \nFig. 1 Structure and functions of EGFR. A EGFR exon boundaries and associated extracellular, transmembrane, and intracellular protein domains. \nEGFR is involved in multiple signaling pathways and regulates numerous cell functions. B The tyrosine kinase domain of EGFR and the activation \nor inactivation of the catalytic domain. C EGFR domains and the molecular mechanisms of acquired resistance. The intracellular domain contains a \njuxtamembrane domain, tyrosine kinase domain, and multiple C-terminal tyrosine residues. Multiple mutations within the tyrosine kinase domain \nare associated with resistance and sensitivity to EGFR-TKIsPage 3 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nThird-generation EGFR-TKIs that target EGFR-TKI-\nsensitive mutations and the T790M mutation have been \ndeveloped [10].\nUnfortunately, drug resistance caused by less-common \nmutations in the EGFR gene and components of signal \ntransduction pathways continues to emerge. In addition \nto common secondary (T790M) and tertiary (C797S) \nmutations, other EGFR mutations (such as the L718Q, \nL796S, and L792H mutations and the exon 20 inser -\ntion), MET amplification, phosphatidylinositol 4,5-bis -\nphosphate 3-kinase catalytic subunit alpha (PIK3CA) \nmutations, HER2 amplification, oncogene fusions, and \nalterations in cell cycle-related genes have been observed \n[11] (Fig.\xa0 1C). There is an urgent need for better strate -\ngies to combat the inevitable molecular-targeted drug \nresistance associated with third-generation inhibitors. \nThis review aims to provide a comprehensive overview of \nthe mechanisms of resistance to third-generation EGFR-\nTKIs and to explore new insights and strategies for over -\ncoming acquired resistance.\nThird‑generation EGFR‑TKIs and\xa0drug resistance \nmechanisms\nThe development of\xa0third‑generation EGFR‑TKIs\nThe first-generation EGFR-TKIs form hydrogen bonds \nwith Met793 in the ATP binding pocket of EGFR and \nreversibly compete with ATP for binding. Drug resistance \noccurs due to the EGFR T790M mutation (Thr790 in the \nhydrophobic ATP binding site encoded on exon 20 is \nreplaced by methionine), subclonal selection (of a geneti -\ncally resistant clone), and rare EGFR mutations (such \nas G719X, S768I, and L861Q). Thereafter, the develop -\nment of second-generation EGFR-TKIs was reported; \nthese inhibitors have the same quinazoline scaffold as \nfirst-generation EGFR-TKIs, but the side chain can irre -\nversibly bind to Cys797 to inhibit the tyrosine kinase \nactivity of EGFR. For example, the anilinoquinazoline derivative forms hydrogen bonds with the backbone \nof Met793 in the hinge region and interacts with the \nhydrophobic region. The acrylamide group binds cova -\nlently to Cys797 in the active conformation of EGFR, the \nfuranyl group is exposed to solvent, and the 3-chloro-\n4-fluorophenyl group is situated next to the gatekeeper \nresidue [12–14]. However, mutations such as T790M still \nemerge upon treatment with second-generation EGFR-\nTKIs, which have limited selectivity against WT-EGFR, \nresulting in serious side effects [15]. Fortunately, third-\ngeneration covalent inhibitors that bind irreversibly to \nthe target and are mutation-selective have been devel -\noped. These compounds were designed based on a new \naminopyrimidine scaffold and show preferable biological \nactivities [16]. Early clinical trials have proven that these \nthird-generation EGFR-TKIs are effective in patients \nwith double-mutated tumors (EGFR L858R/T790M or \nex19del/T790M) and have high selectivity for mutant \nEGFR, thereby eliminating the side effects in the skin \nand gastrointestinal system associated with the nonselec -\ntive inhibition of WT-EGFR [17]. For example, the crys -\ntal structures of rociletinib (CO-1686) in complex with \nEGFR T790M and EGFR L858R have been published; in \nEGFR T790M, the anilinopyrimidine group of rociletinib \nforms hydrogen bonds with the Met793 amide and the \ncarbonyl backbone, whereas in EGFR L858R, hydropho -\nbic interactions between rociletinib and the protein were \ndue to hydrogen bonds between nitrogens in the pyrimi -\ndine group and between the fluoromethyl and Thr790. \nIn addition, the acrylamide group in rociletinib cova -\nlently binds to Cys797 in the DFG-in/αC-in active con -\nformations [18]. The specificity for EGFR T790M may \nstem from hydrophobic interactions between the large \nmethionine in mutant EGFR and pyrimidines. Drugs that \nhave been approved for marketing include osimertinib \n(US), almonertinib (China), lazertinib (South Korea), and \nalflutinib (China) (Fig.\xa02).\nFig. 2 Development of third-generation EGFR-TKIsPage 4 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nMechanisms of\xa0resistance to\xa0third‑generation EGFR‑TKIs\nDue to the covalent bond between the acrylamide \n(Michael acceptor) of third-generation EGFR-TKIs \nand the active thiol in the EGFR kinase domain, highly \nselective inhibitory activity has been achieved by target -\ning Cys797 and irreversible binding EGFR; thus, these \ncompounds show excellent antitumor activity. Targeted \ntherapy for patients with EGFR T790M and EGFR-acti -\nvating mutations showed good efficacy in both first- and \nsecond-line settings. In patients who developed resist -\nance to third-generation EGFR-TKIs as first-line ther -\napy, genetic changes such as MET amplification, EGFR \nC797X mutation, PIK3CA amplification and mutation, \nHER2 amplification and mutation, K-RAS mutation, and \nBRAF mutation, as well as changes in cell cycle-related \ngenes and oncogene fusions, have been reported, but no \nT790M mutations have been detected. The mechanism of resistance to second-line therapy is more compli -\ncated. Acquisition or deletion of the T790M mutation \nhas been detected in patients [19], and other EGFR muta -\ntions (such as L718Q, L796S, L792H, and exon 20 inser -\ntion) have also been observed (Fig.\xa0 1B). In addition, the \nmechanisms of acquired resistance to third-generation \nEGFR-TKIs include alternative pathway activation and \nhistologic and phenotypic transformation (Fig.\xa0 3); the \ndetails will be discussed in the following sections.\nPrimary/intrinsic resistance\nThe differential sensitivity of TKIs to different EGFR \nmutations is a cause of primary drug resistance. In \nNSCLC patients, the in-frame deletion of exon 19 \n(ex19del) and the L858R point mutation in exon 21 \nare the most common somatic mutations, occur -\nring in approximately 80% of cases. During EGFR-TKI \nFig. 3 Molecular mechanisms of acquired resistance. The mechanisms include target gene modification, parallel alternative pathway activation, \ndownstream pathway activation, and histological/phenotypic transformation. Both amplification and mutation of receptor tyrosine kinases (RTKs) \ncan induce downstream survival signaling pathways. Moreover, direct overexpression and/or mutation of components of downstream pathways \ncan contribute to acquired resistance by promoting cancer cell survivalPage 5 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \ntreatment, patients with longer median survival have \npresented with more than 20 unique deletions of exon \n19. Intrinsic drug resistance can all be triggered by \nother nonclassical sensitizing mutations (mainly exon \n20 insertion) and inherent secondary genetic changes. \nDrug-resistant clones (for example, T790M) may \nalready exist within the cancer cell population, leading \nto drug resistance during treatment [20]. Some studies \nhave found that in nearly 1% of lung cancer patients, \n2–3 simultaneous driver mutations can be detected \nbefore treatment. Some molecular and genetic changes \nhave been reported to relate to intrinsic drug resist -\nance, such as the lack of K-RAS/phosphatase and tensin \nhomolog (PTEN) expression. These preexisting molec -\nular and genetic alterations can stimulate the Ras/Raf/\nMEK/ERK and PI3K/AKT downstream pathways to \npromote cancer progression [21].\nBIM deletion polymorphism\nBIM is a proapoptotic member of the B-cell lym -\nphoma-2 (Bcl-2) family [22]. Recent studies showed \nthat lung cancer cells with the BIM deletion polymor -\nphism and EGFR mutation are resistant to third-gen -\neration EGFR-TKIs, suggesting that the BIM deletion \npolymorphism has potential as a biomarker to predict \nthe efficacy of third-generation EGFR-TKIs in patients \n[22].\nEGFR exon 20 insertion\nThe molecular mechanism of drug resistance caused by \nthe exon 20 insertion is not fully understood. Eck et\xa0 al. \n[23] hypothesized that this mutation prevents binding to \nEGFR-TKIs due to the addition of residues to the N-lobe \nof EGFR. The crystal structure of EGFR exon 20 with the \nD770_N771insNPG insertion shows an unchanged ATP \nbinding pocket and a rigid active conformation, leading \nto steric hindrance of the drug binding pocket and resist -\nance to EGFR-TKIs.\nAcquired resistance\nAcquired drug resistance refers to the process by which \ntumor cells with prior sensitivity to treatment circum -\nvent the inhibitory effects of drugs by changing their \nmetabolic pathways. The mechanisms of acquired resist -\nance to third-generation EGFR-TKIs can be divided into \nEGFR-dependent resistance and EGFR-independent \nresistance [24].EGFR‑dependent drug resistance mechanisms\nReappearance of\xa0an\xa0EGFR mutation\nC797S mutation\nOne point mutation of EGFR (C797S) involves the \nreplacement of Cys797 within the ATP binding site \n(exon 20) with serine [25]. Osimertinib binds covalently \nand irreversibly to EGFR T790M by interacting with \nCys797. When the C797S mutation occurs, the osi -\nmertinib binding efficiency decreases [10], resulting in \ntumor resistance to all third-generation EGFR-TKIs.\nG796R/D mutation\nThe G796R mutation has been detected in cancer \npatients who received treatment with a third-gen -\neration EGFR-TKI. Molecular docking predictions \nrevealed that G796R sterically hinders the covalent \nbinding of osimertinib. Because the bulky side chain \nand hydrophilic group hinder the binding of osimerti -\nnib to the hydrophobic region, the change in binding \nenergy renders binding unfavorable. Compared with \nsamples containing the double-mutant EGFR L858R/\nT790M, those harboring the triple-mutant EGFR \nL858R/T790M/G796R are 110 times more resistant \nto osimertinib [26]. G796D was reported for the first \ntime in osimertinib-resistant NSCLC patients. In\xa0vitro \nstudies have shown that the G796D mutation causes \na 50-fold increase in the growth inhibitory 50%  (GI50) \nvalue of osimertinib. Structural modeling showed that \nthe side chain of the mutated G796D residue collides \nwith the surface of osimertinib, resulting in steric hin -\ndrance and energy repulsion and ultimately the loss of \nbinding affinity [27].\nL792 mutation\nThe mutations at Leu792 include L792F, L792Y, and \nL792H. Structural prediction revealed that these muta -\ntions introduce a benzene ring or imidazole ring to the \nside chain of the residue at 792, which spatially dis -\nrupts the orientation of osimertinib, thereby potentially \naffecting the binding of osimertinib to the EGFR ATP \nbinding site [28].\nM766Q mutation\nThe homology simulation with the T790M and M766Q \ndouble mutant showed that M766Q seems to position \nT790M in the inhibitor binding site, thereby weakening \nosimertinib binding [29].\nMutations in\xa0exon 18\nEGFR L718Q/V EGFR L718Q was reported for the first \ntime in a cell model of resistance to third-generation \nEGFR-TKIs. Subsequent studies have shown that NSCLC Page 6 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nwith EGFR L858R/T790M/L718Q is resistant to all EGFR-\nTKIs, but that with only L858R/L718Q remains sensitive \nto afatinib [30]. The crystallographic model revealed that \nthe L718Q mutation reduces the efficiency of the forma -\ntion of covalent bonds between the acrylamide warhead \nand the Cys797 thiol group, thus interfering with the irre -\nversible binding of osimertinib [31, 32]. In addition, L718V \nresistance mutations in the kinase domain of EGFR have \nbeen detected, and these may interfere with the binding \nof osimertinib to the kinase domain [33]. Of note, EGFR \nL718Q/V is still sensitive to afatinib [32].\nEGFR G724S The G724S mutation in the ATP binding \nloop enriches this loop in glycine, which can lead to the \ndevelopment of resistance to EGFR-TKIs by changing the \nprotein structure, enhancing ATP affinity, and stabilizing \nactivating mutations [34]. However, this mutation does \nnot lead to resistance to second-generation EGFR inhibi -\ntors [34].\nCompound mutations\nA compound mutation refers to the simultaneous detec -\ntion of two or more different types of EGFR mutations in \npatient cancer cells [35]. The impact of compound muta -\ntions on EGFR-TKI sensitivity is listed in descending \norder: double classic mutations, compound mutations \ninvolving classic mutations and rare mutations, and com -\npound mutations of only rare mutations [36, 37]. These \nEGFR mutations caused by treatment with third-genera -\ntion EGFR-TKIs confer resistance to irreversible pyrimi -\ndine TKIs but not to quinazoline EGFR inhibitors [38].\nT790M reduction or\xa0deletion\nDeletion of T790M may result from third-generation \nEGFR-TKI treatment or may be one of the reasons \nfor drug resistance related to tumor heterogeneity. In \npatients with EGFR T790M, resistance mechanisms are \noften associated with the C797S mutation or aberrant \nactivation of compensatory pathways, whereas patients \nwith the deletion of T790M typically exhibit different \nresistance mechanisms, most of which are not associated \nwith EGFR signaling pathways [39].\nEGFR amplification\nPiotrowska and colleagues reported EGFR T790M allele \namplification in rociletinib-resistant clones [40]. Nukaga \net\xa0al. found that amplification of the WT allele of EGFR is \nsufficient to mediate resistance to third-generation TKIs. \nThe mechanism of drug resistance may be that EGFR \ngene amplification leads to a relatively low TKI concen -\ntration that is insufficient to exert inhibitory activity [41].EGFR‑independent resistance mechanisms\nNot all patients develop resistance to TKIs through \nEGFR mutation; other pathways of acquiring resistance \nto third-generation EGFR-TKIs include the activation \nof alternative or downstream signaling pathways, epi -\nthelial interstitial resistance, epithelial–mesenchymal \ntransition (EMT), histologic and phenotypic transfor -\nmation, oncogene fusion, and cell cycle-related gene \nabnormalities.\nBypass signal pathway activation\nAbnormal activation of\xa0MET\nThere are two main drug resistance mechanisms caused \nby the abnormal activation of MET: the MET exon 14 \nskipping mutation (METex14) and MET amplification. \nMETex14 leads to the loss of ubiquitin ligase binding \nsites, a reduction in receptor ubiquitination, and per -\nsistent MET activation, resulting in tumor cell survival \nand acquired resistance [42]. After treatment with \nthird-generation EGFR-TKIs, MET gene amplification \ncan promote drug resistance by activating MAPK/ERK, \nwhich is independent of EGFR [43].\nHER2 amplification\nHus et\xa0al. found that H1975 cells expressing HER2D16 \nwere resistant to osimertinib in\xa0 vitro. HER2D16 can \nform a heterodimer with EGFR or a disulfide homodi -\nmer, which activates downstream signaling to achieve \nresistance to osimertinib [44]. HER2D16-driven drug \nresistance occurs in a manner unrelated to the kinase \nSrc. In addition, other mutations in exon 20 of HER2 \nhave been reported, including point mutations (such \nas G776C and L755S) and insertions that cause down -\nstream activation [45, 46]. HER2 mutation occurs in \napproximately 2–4% of NSCLC cases, mostly in lung \nadenocarcinoma (LUAD) [47]. In NSCLC, HER2 onco -\ngenic amplification occurs in approximately 3% of cases \nwithout EGFR-TKI treatment and accounts for approx -\nimately 10% of cases with EGFR-TKI resistance [48].\nAXL activation\nAXL is an RTK that regulates cell survival, proliferation, \nmetastasis, and other cellular functions. Abnormalities \nin the AXL gene can generate acquired resistance to \nTKIs by activating relevant downstream signaling path -\nways. Osimertinib was found to trigger AXL activation \nby closing the negative feedback loop with SPRY4, thus \ntriggering inherent osimertinib resistance [49].\nOverexpression of\xa0HGF\nHepatocyte growth factor (HGF) is the ligand of the \nproto-oncogene c-Met; it can trigger MET activation Page 7 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nthrough EGFR bypass signaling and induce lung can -\ncer resistance to EGFR-TKIs. Yano et\xa0al. [50] found that \nhigh expression of HGF was related to the acquired \nand intrinsic drug resistance to EGFR-TKIs in patients \nwith lung cancer. Tumor specimens from patients \nwith acquired drug resistance showed high expression \nof HGF in the context of MET amplification and the \nT790M mutation.\nFibroblast growth factor receptor (FGFR) signaling\nFGFR is a transmembrane RTK. Studies have shown that \nFGFR1 is amplified and fibroblast growth factor 2 (FGF2) \nmRNA levels are increased in patients with osimertinib \nresistance, suggesting that the FGFR2-FGFR1 autocrine \nloop may be related to drug resistance [51]. Patients with \nthe T790M mutation have been reported to show disease \nprogression after treatment with osimertinib and nilo -\ntinib. The FGFR3-TACC3 fusion was detected in ctDNA \n[52, 53]. These findings suggest that abnormalities in the \nFGFR signaling pathway may underlie the mechanism of \nacquired resistance to third-generation EGFR-TKIs.\nInsulin‑like growth factor receptor 1 (IGF1R)\nIGF1R, a transmembrane heterotetrameric protein \nencoded by the gene located on chromosome 15q26.3, is \ninvolved in promoting the growth of tumor cells. Abnor -\nmal activation of IGF1R leads to EGFR-TKI resistance \n[54].\nAurora kinases (AURKs)\nAURKs are an important category of enzymes within the \nserine/threonine kinase family consisting of three mam -\nmalian isoforms: Aurora kinase A (AURK A), AURK B, \nand AURK C [55, 56]. AURK A and AURK B are highly \nexpressed in dividing cells and play important roles in \nmitotic progression. Mammalian AURK A and AURK B \nshare approximately 71% similarity in the carboxy-termi -\nnal catalytic domain [57]. Aberrant expression of AURK \nA and AURK B is involved in a broad range of solid can -\ncers and is associated with adverse prognosis and drug \nresistance [58, 59]. In addition, Tanaka et\xa0al. [60] reported \nthat targeting AURK B can prevent and overcome resist -\nance to EGFR inhibitors in lung cancer by enhancing \nBIM- and PUMA-mediated apoptosis.\nDownstream signaling pathway activation\nThe activation of signaling pathways downstream of \noncogenic receptors can regulate cell proliferation, cell \ncycle progression, and cell survival. Therefore, the direct \nregulation of downstream signaling pathway-related fac -\ntors can lead to acquired resistance.K‑RAS mutation\nAn epidemiological meta-analysis found that K-RAS \nmutations are present in NSCLC patients, and all \npatients with K-RAS mutations were resistant to EGFR-\nTKIs [61]. K-RAS mutation is related to activation of \nthe RAS-MAPK pathway. The common K-RAS muta -\ntions include G12S, G12D, G12A, Q61H, and A146T. \nStudies have found that inhibiting mutant K-RAS can \nreduce tumor growth and render NSCLC patients sen -\nsitive to EGFR inhibitors [62].\nBRAF (v‑RAF murine sarcoma viral oncogene homologue B1) \nmutation\nBRAF is a serine/threonine protein kinase that plays \na key role in the MAPK/ERK pathway, including in \nEGFR/RAS/RAF signal transduction. BRAF can regu -\nlate cell survival, proliferation, differentiation, and \napoptosis, as well as tumor induction. Many BRAF \nmutations (G469A, V600E, and V599E) have been \nfound in cancer, including lung cancer [63]. Ohashi \net\xa0 al. [64] reported that in patients with lung cancer, \nBRAF mutations can induce acquired resistance to \nEGFR-TKIs. Preclinical data showed that the BRAF \nV600E mutation has a strong association with resist -\nance to the third-generation EGFR-TKI osimertinib in \npatients with T790M-mutated LUAD.\nPI3K/AKT/mTOR\nPIK3CA is a driver gene of LUAD. Mutation of PIK3CA \ncan promote tumor cell invasion and increase the activ -\nity of downstream PI3Ks. Studies have shown that \nPIK3CA amplification or mutation (including E453K, \nE545K, and H1047R) may occur in patients with osi -\nmertinib resistance [52, 65]. Increased PI3K activity \nleads to the activation of various downstream kinases, \nthereby increasing PI3K/AKT/mTOR pathway activ -\nity in the absence of coupling to upstream EGFR \nphosphorylation.\nSTAT3 activation\nSTAT proteins, especially STAT3, are key down -\nstream signal sensors of EGFR activation. In studies \non NSCLC, Zhao et\xa0al. [66] discovered the clinical sig -\nnificance of JAK2/STAT3 in angiogenesis. Chaib et\xa0 al. \n[67] found that osimertinib treatment activates not \nonly STAT3 but also SrcYAP1 signaling, which may act \ndownstream of IL-6 to promote disease progression.\nLoss of\xa0PTEN\nPTEN is a tumor suppressor gene that encodes a pro -\ntein with lipid phosphatase activity and thus regulates \ncellular protein phosphatase activity. PTEN has dual Page 8 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nantitumor effects and is a key component of many sign -\naling pathways in the body. If mutation or deletion of \nthe PTEN gene or downregulation of PTEN expres -\nsion can reduce or eliminate its antitumor activity [68], \nloss of PTEN leads to hyperactivation of the PI3K/\nAKT signaling pathway and resistance to EGFR-TKIs, \nincluding osimertinib.\nHyperactivation of\xa0activated Cdc42‑associated kinase 1 \n(ACK1)\nHyperphosphorylation of ACK1 and the subsequent acti -\nvation of antiapoptotic signaling through the AKT path -\nway are associated with resistance to third-generation \nEGFR-TKIs [69].\nc‑Myc gene\nThe c-Myc gene is an important member of the MYC \ngene family. The c-Myc gene can induce cells to prolif -\nerate indefinitely and can promote cell division; these \nactivities are related to the occurrence and develop -\nment of various types of cancer. Studies have shown that \nc-Myc levels are substantially elevated in different EGFR-\nmutant NSCLC cell lines with acquired resistance to the \nthird-generation EGFR-TKI osimertinib compared with \nthe corresponding parental cell lines; moreover, these \nincreased levels cannot be reduced by osimertinib. Con -\nsistently, c-Myc levels are elevated in the majority of \nEGFR-mutant NSCLC tissues from patients who relapsed \non EGFR-TKI treatment compared with the correspond -\ning baseline c-Myc levels prior to treatment [70]. These \nfindings indicate that c-Myc mediates the therapeutic \nefficacy of third-generation EGFR-TKIs and the develop -\nment of acquired resistance to these TKIs.\nOther mechanisms\nEpithelial–mesenchymal transition (EMT)\nIn EMT, cancer cells lose their epithelial properties \nthrough the loss of E-cadherin, leading to increased \nvimentin expression and transformation into a mesen -\nchymal phenotype. A previous study found that osimerti -\nnib-resistant H1975 cells have EMT characteristics in the \nabsence of other EGFR mutations [71]. EMT is a coor -\ndinated process involving multiple regulatory factors, \nsuch as EMT-induced transcription factors (EMT-TFs), \nnoncoding RNAs (ncRNAs), and various extracellular \nsignals. EMT-TFs play an important role in all stages of \nEMT; the most well-known EMT-TFs are members of \nthe SNAIL, ZEB, and TWIST families. Many studies have \nshown that SLUG and SNAIL overexpression can induce \ndrug resistance [72].miRNAs and\xa0EMT\nLong noncoding RNAs (lncRNAs) and microRNAs \n(miRNAs) play important roles in regulating EMT and \nTKI resistance. Although most miRNAs have been \nfound to inhibit EMT, some have activity that pro -\nmotes EMT, including miR-21 and miR-155 [73, 74]. \nSome miRNAs can promote TKI resistance by activat -\ning the PI3K/AKT/mTOR signaling pathway; for exam -\nple, miR-21 and miR-23a can target PTEN and activate \nAKT, leading to resistance to EGFR-TKIs [75, 76].\nEpigenetic alterations\nEpigenetic modifications involved in cancer initiation \nand progression include changes in DNA methylation \npatterns and histone modifications. Epigenetic changes \nare common in the development and progression of \nlung cancer [77]. Studies have shown that epigenetic \ndisorders can make cancer patients susceptible to \nacquired resistance to EGFR-TKIs [78].\nOncogene fusion\nThe AURA-3 and FLAURA trials showed that onco -\ngene fusion might be one mechanism of osimertinib \nresistance; the identified fusions included transform -\ning growth factor receptor (TGFR)-transforming acidic \ncoiled-coil protein 3 (TACC3), neurotrophic receptor \ntyrosine kinase 1 (NTRK1)-thrombopoietin mimetic \npeptide 3 (TMP3), ERC1-RET, SPTBN1-ALK, coiled-\ncoil domain-containing protein 6 (CCDC6)-RET, \nGOPC-ROS1, AGK-BRAF, NCOA4-RET, ESYT2-BRAF, \nand echinoderm microtubule-associated protein-like 4 \n(EML4)-ALK. Oncogene fusions can coexist with the \nEGFR C797S mutation, MET amplification, and BRAF \nmutation [79].\nCell cycle‑related gene abnormalities\nRecent studies have shown that changes in cell cycle-\nrelated genes, including the CDKN2A E27fs muta -\ntion, cyclin D (CCND) amplification, cyclin-dependent \nkinase 4/6 (CDK4/6) amplification, and cyclin E1 \n(CCNE1) amplification, can cause resistance to third-\ngeneration EGFR-TKIs [65].\nHistologic and\xa0phenotypic transformation\nHistopathological transformation to small cell lung \ncancer (SCLC) from NSCLC has been reported as a \nmechanism of acquired resistance to EGFR-TKIs in \n3–15% of patients [80– 83]. Transformed SCLC mainly \noccurs in Asian patients with adenocarcinoma harbor -\ning EGFR-TKI-sensitive mutations (such as the EGFR \nex19del/T790M mutation) who are nonsmokers. The \nwidely accepted hypothesis for this transformation Page 9 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nposits that adenocarcinoma and SCLC originate from \ntype II alveolar cells. RB1 and TP53 mutations might be \ninvolved in SCLC transformation but are not sufficient \nfor the induction of complete transformation. Addi -\ntional genomic alterations, including those that acti -\nvate the PI3K/AKT family and downregulate NOTCH \nsignaling and those affecting the MYC and SOX fami -\nlies, AKT pathway activation and other molecules, also \nparticipate in the transformation from EGFR-mutant \nNSCLC. However, the precise mechanisms in other \ncases are unclear [84]. In addition, squamous cell trans -\nformation was recently identified as a mechanism of \nacquired EGFR-TKI resistance that occurs in approxi -\nmately 15% of patients who received osimertinib as \nboth first- and second-line therapy. Similar to the case \nin SCLC transformation, the primary EGFR mutation is \npreserved in squamous cell transformation [85].\nImmune escape\nEGFR is expressed in different hematopoietic cell types, \nincluding macrophages, monocytes, and certain T-cell \nsubsets. Therefore, it is likely that EGFR inhibitors can \ninterfere with the function of these leukocytes. Immune \ncheckpoint inhibitors (ICIs) have adverse effects and \npoor efficacy in patients with an EGFR mutation or a sec -\nondary T790M mutation, largely because of low tumor \nmutational burden and a noninflamed tumor micro -\nenvironment [86–88]. A previous study showed that \nsecreted phosphoprotein 1 (SPP1) promotes macrophage \nM2 polarization and PD-L1 expression in LUAD, which \nmay influence the response to immunotherapy. SPP1 lev -\nels might be a useful marker of immunosuppression in \npatients with an EGFR mutation and could provide thera -\npeutic insight [89]. In addition, HGF, MET amplification, \nand EGFR T790M lead to the upregulation of PD-L1 \nexpression in NSCLC and promote immune escape by \ntumor cells through different mechanisms mediated by \nthe PI3K-Akt, MAPK, and NF-κB pathways [90].\nStrategies for\xa0overcoming third‑generation EGFR‑TKI \nresistance\nFourth‑generation EGFR‑TKIs: overcoming the\xa0L858R/T790M \nand\xa0C797S resistance mutations\nThird-generation EGFR-TKIs had the potential for \nremarkable achievements, if not for the numerous \nmutations. The C797S mutation, which is a covalent \nanchor mutation, is located in the ATP binding site of \nthe EGFR tyrosine kinase domain. This missense muta -\ntion in exon 20 at position Cys797 blocks the ability of \nthird-generation EGFR-TKIs to form a covalent bond \nin the ATP binding region, with a consequent decrease \nin the binding affinity between EGFR and an EGFR-\nTKI [91]. The combination of the C797S mutation with exon 19 deletion, L858R mutation, or T790M muta -\ntion was reported both in\xa0vitro and in\xa0vivo [91]. Studies \nhave shown that drug-resistant lung cancer cells with \ntwo mutations (EGFR-activating mutation/C797S) are \nsensitive to first- and second-generation EGFR-TKIs. \nHowever, lung cancer cells with three mutations (EGFR-\nactivating mutation/T790M/C797S) show resistant to \nthird-generation EGFR-TKIs if the C797S and T790M \nmutations are both in the trans conformation. None -\ntheless, these cells are still sensitive to the combination \nof first- and third-generation EGFR-TKIs [92]. Of note, \nif C797S and T790M are mutated in the cis conforma -\ntion, the cells show resistance to all existing EGFR-TKIs \n(either alone or in combination) [93]. The resistance to \nthird-generation EGFR-TKIs caused by the trans-C797S \nmutation can be overcome by drugs targeting different \nkinase binding sites, including allosteric inhibitors, ATP-\ncompetitive inhibitors, and “dual-site” inhibitors that \noccupy both the ATP binding site and an allosteric site.\nAllosteric inhibitors\nEGFR has three binding sites: an inactive site, a competi -\ntive ATP binding site, and an allosteric site. Ligands and \ndrugs cannot bind the inactive site. Recent studies have \nmostly focused on either ATP-competitive inhibitors tar -\ngeting the ATP binding site or molecules that bind the \nallosteric site, which causes a conformational change in \nthe protein that inhibits the signaling cascade [94]. To \novercome the resistance of EGFR-TKIs mediated by the \nT790M and C797S mutations and to further identify and \nexplore compounds that bind outside the ATP binding \ndomain of EGFR, researchers have pursued the devel -\nopment of allosteric inhibitors, and this appears to be a \npromising strategy. The newly developed fourth-gen -\neration mutant-selective allosteric inhibitors can over -\ncome the T790M and C797S mutations that develop in \nresponse to third-generation EGFR-TKIs by binding to \nsites outside the ATP binding pocket of EGFR.\nFig. 4 Screening hit 1a (1) from a phenotypic screen of NSCLC \ncell lines. A Structure of 1a (1). B X-ray crystal structure of 1a (1) in \ncomplex with c-Src-DM (PDB code: 5D12)Page 10 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nThrough molecular phenotypic screening, Engel et\xa0 al. \nobtained quinazoline compound 1a (1), which specifi -\ncally inhibits the drug-resistant H1975 cell line (L858R/\nT790M); further modification addressed the problem of \noff-target activity (nonspecific inhibition). X-ray crystal -\nlography verified that compound 1a (1) fits well in the \ntyrosine kinase domain of c-Src [95] (Fig.\xa04).\nJia et\xa0 al. conducted counter-screening of active com -\npounds against WT-EGFR and discovered the first non-\nATP-competitive allosteric EGFR L858R/T790M/C797S \ninhibitor based on the thiazolamide scaffold (EAI001, \n2) [96]. The X-ray crystal structure of EAI001 (2) in \ncomplex with EGFR T790M shows that EAI001 (2) can \nbind to the allosteric site of this receptor in the form of \na “three-bladed propeller, ” partly due to the outward \ndisplacement of the C-helix in the inactive conforma -\ntion of the kinase. The hydrophilic side chain of the WT \ngatekeeper residue (Thr) cannot adapt to the thiazole of \nEAI001; therefore, there is no favorable interaction. The \nthiazole of EAI001 closely interacts with the hydrophobic \nside chain of Met790; specifically, aminothiazole group \nof EAI001 directly binds to Met790. The carbonyl oxy -\ngen of the isoindoline-1-one moiety is inserted between \nthe mutant gatekeeper residue (Met) and the active site \nresidue Lys745, forming another hydrogen bond with \nthe ε-amine of the Lys745 side chain. The NH group of \nformamide acts as a hydrogen bond donor for Asp855 \nin the DFG motif. The cationic phenyl group occupies \nthe hydrophobic pocket formed by Met766, Leu777, \nand Phe856. The 1-oxindolinyl group is exposed along \nthe C-helix and extends to the solvent-accessible area. \nThe ATP analog adenylyl imidodiphosphate (AMP-\nPNP) binds the active site cavity in an expected man -\nner. The half maximal inhibitory concentration  (IC50) of \nEAI001 (2) for EGFR L858R/T790M is 24\xa0nmol/L, which \nis lower than that for WT-EGFR  (IC50 > 50\xa0μmol/L). The \n IC50s of EAI001 (2) for EGFR L858R and EGFR T790M \nis 0.75\xa0 μmol/L and 1.7\xa0 μmol/L, respectively. By intro -\nducing ortho-hydroxyl and meta-fluorine atoms on the \nbenzene ring of EAI001 (2), the researchers synthesized \nanother compound, EAI045 (3), that binds more tightly \nthan EAI001 (2) to EGFR [96]. However, EAI045 has a \nmajor drawback: it must be used in combination with \ncetuximab to preserve its efficacy. While EAI045 (3) has \ngood selectivity for WT-EGFR, cetuximab is expected \nto have off-target effects in clinical use. Lee et\xa0 al. [97] \ndesigned the EGFR allosteric inhibitor TREA-0236 (4) \nbased on the structure–activity relationships of EAI045 \n(3). The structure of EAI045 (3) was modified by cycliza -\ntion, wherein the 2-aminothiazole amide was converted \nto quinazoline-4-one. To minimize hematological and \nmethemoglobinemia toxicity and to obtain better safety \nand pharmacokinetic parameters, To et\xa0 al. linked the 5-indole substituent to the isoindolinone of EAI001 (2) \nand obtained a new EGFR allosteric compound, JBJ-02-\n112-05 (5), with an  IC50 of 15\xa0nmol/L for EGFR L858R/\nT790M [98]. Additionally, EAI045 (3) was further opti -\nmized to generate another EGFR allosteric inhibitor, JBJ-\n04-125-02 (6), in which the 2-hydroxy-5-fluorophenyl \nof EAI045 (3) was combined with the phenylpiperazine \non isoindolinone. This compound showed a significantly \nincreased ability to inhibit EGFR L858R/T790M, with \nan  IC50 of 0.26\xa0 nmol/L. Interestingly, combination with \nosimertinib enhanced the efficacy of JBJ-04-125-02 (6) \nand improved the targeting of JBJ-04-125-02 (6) to can -\ncer cells [98], indicating that the combined use of cova -\nlent mutant-selective ATP-competitive inhibitors and \nEGFR allosteric inhibitors may be an effective treatment \nstrategy for patients with EGFR-mutant disease (Fig.\xa0 5). \nEncouraged by the advantages of inhibiting allosteric \nsites in the EGFR tyrosine kinase domain, researchers \nhave extensively designed and optimized allosteric inhib -\nitors for EGFR [98–101], as shown in Table\xa01.\nThe novelty of allosteric sites has attracted the atten -\ntion of researchers, and these sites have become the \nmost promising targets for the development of drugs \nfor NSCLC and other diseases. Fourth-generation \nEGFR-TKIs require further investigation and devel -\nopment so that they are suitable as single-agent drugs \ntargeting EGFR ex19del/T790M/C797S [98]. Allosteric \ninhibitors have now entered the stage of rapid develop -\nment and are expected to enter clinical trials soon, with \nthe goal of benefitting more patients.\nATP‑competitive inhibitors\nATP-competitive inhibitors form one to three hydro -\ngen bonds with amino acids in the hinge region of the \ntarget kinase, thereby mimicking the characteristic \nhydrogen bonds formed by the adenine ring of ATP . \nThis type of inhibitor usually consists of a heterocy -\nclic ring system that occupies the purine binding site, \nwhere it acts as a side chain scaffold that occupies the \nadjacent hydrophobic regions I and II. A high physio -\nlogical or intracellular concentration of ATP may block \nthe phosphotransferase activity of the target. The size \nof the amino acid side chain at the gatekeeper residue \ndetermines the relative accessibility of the hydrophobic \npocket near the ATP binding site. To overcome drug \nresistance related to triple-mutant EGFR, it is particu -\nlarly crucial to develop new ATP-competitive inhibi -\ntors based on structural design and optimization. Many \nATP-competitive inhibitors have been reported; below, \nwe summarize recent ATP-competitive inhibitors that \ncan overcome the resistance to third-generation EGFR \ninhibitors (Table\xa02 ) [102– 119].Page 11 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \n“Dual‑site” inhibitors: occupying both\xa0the\xa0ATP binding site \nand\xa0the\xa0allosteric site\nBased on the non-ATP-competitive EGFR L858R/\nT790M/C797S inhibitor EAI001 reported by Jia et\xa0 al., \nthe more potent compound EAI045 (3 ) was obtained \nthrough structural optimization [96]. EAI045 (3 ) \nbinds to the allosteric site created by the outward displacement of the αC helix of EGFR, located next to \nthe ATP binding pocket. Facilitated by molecular dock -\ning, researchers developed a series of new compounds \nthat noncovalently occupy both the EGFR ATP bind -\ning site and the allosteric site; these fourth-generation \nreversible EGFR inhibitors have improved binding \naffinity for EGFR L858R/T790M/C797S, effectively \nFig. 5 Chemical structures and structure–activity relationships of allosteric inhibitors. A The rational design of TERA-0236 (4), JBJ-02-112-05 (5), and \nJBJ-04-125-02 (6) and their inhibitory activities against EGFR. B X-ray cocrystal structure of EGFR with EAI001 (2) (PDB code: 5D41). C X-ray cocrystal \nstructure of EGFR with EAI045 (3). D X-ray cocrystal structure of EGFR with EAI045 (in the active and inactive states) (PDB code: 5ZWJ). E X-ray \ncocrystal structure of EGFR with JBJ-04-125-02 (6) (PDB code: 6DUK)Page 12 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \ncompete with ATP , and further overcome resistance to \nthird-generation EGFR inhibitors.\nThe compound vandetanib (30) [120] is a known \nEGFR inhibitor that shows moderate efficacy against \nEGFR L858R/T790M/C797S, with an  IC50 value of \n369.2\xa0nmol/L. Via molecular docking simulation, Li et\xa0al. found that vandetanib can extend to the EGFR ATP bind -\ning pocket (gscore =  − 8.2\xa0kcal/mol). The docking model \nof vandetanib with EGFR T790M/V948R shows that the \nphenyl group of vandetanib binds the ATP binding site \nof EGFR, occupying a position such that it resembles \nthe thiazole moiety of EAI001 (2). EAI001 (2) binds as a Table 1 EGFR allosteric inhibitors\nCompound (reference) Structure Activity Interaction with \nEGFR allosteric \nsite\n7, DDC4002 [100] EGFRL858R/T790M\nIC50 = 10 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 59 nmol/LPhe856\n8 [100] EGFRL858R/T790M/C797S\nIC50 = 19 nmol/LPhe856\n9 [100] EGFRL858R/T790M/C797S\nIC50 = 23 nmol/LPhe856\n10 [100] EGFRL858R/T790M/C797S\nIC50 = 13 nmol/LPhe856\n11 [100, 101] EGFRL858R/T790M\nIC50 = 1 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 5 nmol/LPhe856\n12 [100, 101] EGFRL858R/T790M\nIC50 = 3 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 4 nmol/LPhe856Page 13 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nTable 2 ATP-competitive EGFR inhibitors\nCompound (reference) Structure Enzymatic activity Biological activity DMPK profile\n13, JND3229 [102] EGFRL858R/T790M/C797S\nIC50 = 5.8 nmol/LBaF3-EGFRL858R/T790M/C797S\nIC50 = 510 nmol/L\nBaF3-EGFRDel19/T790M/C797S\nIC50 = 320 nmol/LN/A*\n14 [103] EGFRWT\nIC50 = 16 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 88 nmol/LA431\nGI50 = 3600 nmol/L\nH1975\nGI50 = 140 nmol/LN/A\n15 [104] EGFRL858R/T790M/C797S \n IC50 = 8 nmol/LA431\nEC50 = 4000 nmol/L\nH1975\nEC50 = 400 nmol/LN/A\n16 [105, 106] EGFRL858R/T790M/C797S\nIC50 = 630 nmol/LH1975\nIC50 = 1200 nmol/LLiver microsomes (Human):\nt1/2 (min) = 66.6, CLint (mL/\nmin/kg) = 20.8\n17 [107] EGFRWT\nIC50 > 1000 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 27.5 nmol/LBaF3-EGFRL858R/T790M/C797S\nIC50 = 662 nmol/LT1/2 (rat, minutes) = 8.36,\nCLint (mL/min/kg) = 297.12Page 14 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nTable 2 (continued)\nCompound (reference) Structure Enzymatic activity Biological activity DMPK profile\n18 [108] EGFRL858R/T790M/C797S\nIC50 = 7.2 nmol/LHCC827\nIC50 = 44 nmol/L\nH1975\nIC50 = 400 nmol/LN/A\n19 [109, 110] EGFRL858R/T790M/C797S\nIC50 = 18 nmol/LHCC827\nIC50 = 0.88 nmol/L\nH1975\nIC50 = 200 nmol/L\nA549\nIC50 = 2910 nmol/L\nA431\nIC50 > 10,000 nmol/LN/A\n20 [111] EGFRL858R/T790M/C797S\nIC50 = 8.5 nmol/LEGFR CTG \nEC50:\nHCC827\nIC50 < 14 nmol/L\nH1975\nIC50 = 51 ± 19 nmol/L\nA431\nIC50 = 1675 ± 402 nmol/L\nA549\nIC50 = 3700 nmol/L\nH358\nIC50 = 3700 nmol/LIn vivo PK (mice, IP , 20 mg/kg):\nAUC free (h∙ng/mL) = 8.6\nt1/2 (h) = 1.2\nCmax,free  (μmol/L) = 0.012\n21 [112] EGFRL858R/T790M/C797S\nIC50 = 3.1 nmol/LH1975\nIC50 = 0.12 ± 0.09 μmol/L\nBaF3-EGFRL858R/T790M/C797S\nIC50 = 0.29 μmol/L\nBaF3-EGFR19D/T790M/C797S \n IC50 = 0.31 μmol/LN/APage 15 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nTable 2 (continued)\nCompound (reference) Structure Enzymatic activity Biological activity DMPK profile\n22 [113] EGFRWT\nIC50 > 1000 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 218.3 nmol/L\nEGFRDel19/T790M/C797S\nIC50 = 15.3 nmol/LH1975\nIC50 = 16,180 nmol/L\nA431\nIC50 = 20,480 nmol/L\nBaF3-EGFRDel19/T790M/C797S\nIC50 = 8510 nmol/LN/A\n23 [114] EGFRWT\nIC50 = 430 nmol/L\nEGFRDel19/T790M/C797S\nIC50 = 0.2 nmol/LBaF3-EGFRWT\nIC50 = 1000 nmol/L\nBaF3-EGFRDel19\nIC50 = 180 nmol/L\nBaF3-EGFRDel19/T790M\nIC50 = 99 nmol/L\nBaF3-EGFRDel19/T790M/C797S\nIC50 = 63 nmol/LN/A\n24 [115] N/A Biochemical potency:\nBaF3 cells  IC50 < 100 nmol/L\nAntiproliferative activity:\nBaF3 cells  IC50 < 100 nmol/LN/A\n25 [116] N/A Antiproliferative activity:\nPC-9-EGFRL858R/T790M/C797S \n IC50 = 595.7 nmol/L\nPC-9-EGFRDel19/T790M/C797S \n IC50 = 739.9 nmol/L\nA549  IC50 = 2861.7 nmol/L\nBaF3-EGFRWT \n IC50 = 519.82 nmol/L\nBaF3-EGFRL858R/T790M/C797S \n IC50 = 0.16 nmol/L\nBaF3-EGFRDel19/T790M/C797S \n IC50 = 0.23 nmol/LN/A\n26 [117] N/A BaF3-EGFRWT \n IC50 = 540 nmol/L\nBaF3-EGFRL858R/T790M/C797S \n IC50 = 48.2 nmol/L\nBaF3-EGFRDel19/T790M/C797S \n IC50 = 12 nmol/LN/APage 16 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nY-shaped constellation in the allosteric site [121]. Modi -\nfying vandetanib to occupy both the ATP binding site and \nthe allosteric site may be an effective way to improve its \nbiological activity against EGFR L858R/T790M/C797S. \nTo promote occupation of the allosteric site of EGFR, \nthe structure of EAI045 (3) was modified such that the \nhydrophobic group oxyisoindole-2 phenylacetamide was \nintroduced with an amide bond as the linker, generating \ncompound 31. With this compound as a new lead, three \nmoieties, namely the allosteric targeting region, the hinge targeting region, and the solvent exposure region, were \nstudied and optimized. Finally, the EGFR L858R/T790M/\nC797S reversible inhibitor compound 32 (Fig.\xa0 6) was \nobtained, with an  IC50 value of 2.2\xa0nmol/L. The docking \nsimulation showed that compound 32 occupies both the \nATP binding region and the allosteric region. In addi -\ntion, it extensively interacts with residues in the allos -\nteric region, the solvent-accessible region, and the hinge \nregion. The phenyl of the Y-shaped group (oxoisoindolin-\n2-phenylacetamide) and Phe856 of the allosteric cavity Table 2 (continued)\nCompound (reference) Structure Enzymatic activity Biological activity DMPK profile\n27 [115] EGFRWT\nIC50 = 7.92 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 0.218 nmol/L\nEGFRDel19/T790M/C797S\nIC50 = 0.16 nmol/LAntiproliferative Activity:\nA431\nIC50 = 154 nmol/L\nBaF3-EGFRDel19/T790M/C797S\nIC50 = 22 nmol/LIn vivo PK (mice, per os, \n15 mg/kg):\nAUC 0-last = 57,037 (nmol/L∙h)\nt1/2 (h) = 10.0\nPlasma (nmol/L), 2 h = 3553\nTumor (nmol/kg), 2 h = 16,667\n28 [118] EGFRWT\nIC50 = 3.8 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 38.1 nmol/LBaF3-EGFRL858R/T790M/C797S\nIC50 < 1000 nmol/LN/A\n29, UPR1444 [119] EGFRWT\nIC50 = 30 ± 4.8 nmol/L\nEGFRL858R/T790M/C797S\nIC50 = 110 ± 33 nmol/LN/A N/A\n*N/A not availablePage 17 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nform π–π stacking interactions. Inside the ATP binding \nregion, hydrogen bonds are formed between the quina -\nzoline ring and the hinge residue Met793. In addition, \nthe piperidine tail is surrounded by the solvent-exposed \nregion. At a concentration of 0.1\xa0 μmol/L, compound \n32 almost completely inhibited the phosphorylation of \nEGFR, showing comparable potency to that of EAI045 \n(3).\nTo further design more potent inhibitors spanning \nboth binding sites, considering the proximity of the ortho \nand allosteric positions, Wittlinger F et\xa0al. compared the binding of the EGFR ATP site inhibitor LN2057 (33) \nwith the allosteric inhibitor EAI045 (3) and found that \nthe 4-fluorophenyl of LN2057 (33) and the thiazole of \nEAI045 (3) had the same binding position [122]. Based \non this, researchers designed and synthesized a series of \ncompounds that combined a large portion of the isomeri -\nzation inhibitor EAI045 (3) with the pyridyl-imidazole \nskeleton. For compound 34, the pyridinylimidazole scaf -\nfold partly binds the 2-fluoro-5-hydroxyphenyl moiety of \nEAI045 (3); 1-oxoisoindoline-2-yl was introduced into \ncompound 35; and 1,3-dioxoisoindoline-2-yl was added \nFig. 6 Chemical structures and structure–activity relationships of dual-site inhibitors: the rational design of compound 32 and its inhibitory activity \nagainst EGFR\n(See figure on next page.)\nFig. 7 Chemical structures and structure–activity relationships of dual-site inhibitors. A Structure-guided design and synthesis of mutant-selective \nlead compounds and their inhibitory activities against EGFR. B Structural superposition of the ATP site binding inhibitor LN2057 (PDB code: 6V6K) \nand the allosteric inhibitor EAI045 (PDB code: 6P1L); C X-ray cocrystal structure of EGFR T790M/V948R with compound 34 (PDB code: 6WA2); D X-ray \ncocrystal structure of EGFR T790M/V948R with compound 36 (PDB code: 6WXN)Page 18 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nFig. 7 (See legend on previous page.)Page 19 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nto generate compound 36 to further explore the struc -\nture–activity relationship of the allosteric site. In addi -\ntion, an N-(4-methoxyphenyl)acrylamide warhead was \nintroduced to produce compound 37, and the influence \nof the C797-targeting capacity of these chimeric com -\npounds, which are expected to form a covalent bond with \nC797, was assessed. The X-ray cocrystal analysis of the \nbinding mode with EGFR T790M/V948R (Fig.\xa0 7) revealed \nthat compounds 34 and 36 bind in the same way. Taking \ncompound 36 as an example, the aminopyridine moiety \nforms a hydrogen bond with the M793 residue in the \nhinge region. The inhibitor is anchored at the ATP bind -\ning site, and the N atom of the imidazole moiety forms \na hydrogen bond with K745, which is essential for the \nstrong reversible binding of the imidazole skeleton. The \nphenylamide bond extending into the allosteric pocket \nis directed toward the T790M mutation, and the N atom \non the amide forms hydrogen bonds with the T854 and \nD855 residues. Despite considerable efforts, the X-ray \ncrystal structure of compound 37 in complex with EGFR \nwas not obtained. Compound 37 was computationally \ndocked to the EGFR T790M/V948R kinase domain, and \nthe result was the same as that for compound 36. The \nmethoxyphenyl acrylamide formed a covalent bond with \nC797. Importantly, no covalent binding of compound 37 \nto the EGFR L858R/T790M/C797S kinase domain was \nobserved, confirming that this compound is a noncova -\nlent inhibitor.\nThe inhibitory activity of the above compounds \nwas tested, and the results showed that compound 34 \nexhibited strong inhibitory activity against all mutants, \nwith  IC50 values of 5–32\xa0 nmol/L, indicating that the \nintroduction of 2-fluoro-5-hydroxyphenyl alone did \nnot increase selectivity. With the introduction of oxy -\nisoindolin-2-yl, the inhibitory activity of compound 35 \ndecreased, but a certain inhibitory effect against EGFR \nL858R was observed. Furthermore, compound 1 inhib -\nited all three EGFR mutants at the low nanomolar range. \nCompound 37 showed a moderate degree of mutation \nselectivity for WT-EGFR, possibly due to the methoxy -\nphenyl acrylamide group. To assess the kinase selectiv -\nity of compound 37, a kinome screen including 335 WT \nkinases was performed; compound 37 exhibited high \nselectivity, with a selectivity score of 0.006 at an inhibi -\ntor concentration of 1\xa0 μmol/L. Next, the antiprolifera -\ntive activity of these compounds was evaluated in Ba/F3 \ncells stably transfected with WT-EGFR, EGFR L858R, \nEGFR L858R/T790M, or EGFR L858R/T790M/C797S. \nAmong the compounds, compound 37 showed an anti -\nproliferative effect in the EGFR L858R and EGFR L858R/\nT790M cell lines, with  IC50 values in the micromolar \nrange in the presence and absence of cetuximab. The \n IC50 value of compound 37 in EGFR L858R Ba/F3 cells (1.2 ± 0.07\xa0μmol/L) was comparable to that of EAI045 (3) \ncombined with cetuximab (840 ± 700\xa0nmol/L). Although \ncompound 37 is potent and selective for kinases, its cellu -\nlar activity is suboptimal. Kinase selectivity was achieved \nby increasing the molecular weight of the lead compound \nand increasing the number of hydrogen bond donors and \nacceptors, but these changes may have produced limited \ncell permeability and effects on cell viability; thus, this \ncompound lacked sufficient activity in cells expressing \nEGFR L858R/T790M/C797S.\nThe selective EGFR inhibitor (compound 37) designed \nand developed in this study can bind to both the ATP site \nand the allosteric site of the EGFR kinase domain. Adding \nallosteric inhibitor elements to the compound skeleton at \nthe ATP binding site contributes to the mutation selec -\ntivity of these compounds. The designed compound 37 \nhas good kinase activity but nonideal cell activity. Future \nresearch and development could optimize the structure \nof this lead compound to further enhance its cellular \nactivity.\nPROTAC technology\nAllosteric EGFR degrader\nResistance to third-generation EGFR-TKIs is a major \nobstacle to clinical targeted therapy. Due to changes \nin the EGFR protein [123], some kinase inhibitors are \nrestricted to the catalytic pocket [124]. A proteolysis-\ntargeting chimera (PROTAC) induces the proteasomal \ndegradation of the target by recruiting it to a specific E3 \nligase. The eradication of EGFR protein from cancer cells \nprovides a promising strategy for overcoming drug resist -\nance. The allosteric EGFR degrader is a heterobifunc -\ntional compound based on allosteric EGFR inhibitors. \nIt includes a small molecule (protein-of-interest (POI) \nligand) that binds the target protein and a small-molecule \nE3 ligase ligand that recruits cereblon (CRBN), von Hip -\npel–Lindau (VHL), cellular inhibitor of apoptosis protein \n1 (cIAP1) or murine double minute 2 (MDM2). After the \naddition of a linker connecting the two parts [125, 126], \nthese chimeras can degrade mutant EGFR without affect -\ning WT-EGFR.\nCompared with classic “occupying” inhibitors, allos -\nteric EGFR degraders can completely eliminate the \nfunction of the target protein, thereby improving the \nphenotypic potency. Moreover, since PROTAC mole -\ncules usually do not require strong binding to targets or \nlong-term retention to achieve protein degradation, the \ndevelopment of drug-induced resistance mutations may \nbe prevented. Compared with kinase inhibitors, PRO -\nTACs have the advantages of activity at lower concentra -\ntions, limited dose-dependent toxicity, and the potential \nto overcome drug resistance and target drug refractory Page 20 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \ndisease [127–132]. These molecules have attracted con -\nsiderable attention from academia and industry and \nhave become an attractive therapeutic strategy in drug \ndiscovery.\nBased on EAI001 (2), a compound that buries deeply in \nthe allosteric pocket [96], Jang et\xa0al. introduced 1-(pyri -\ndin-2-yl)piperazine at the 6 position of isoindolinone \nand synthesized JBJ-07-149 (38), which has an  IC50 value \nof 1.1\xa0nmol/L for EGFR L858R/T790M. In combination \nwith cetuximab, JBJ-07-149 has a half maximal effective \nconcentration  (EC50) of 0.148\xa0 nmol/L for EGFR L858R/\nT790M. However, this compound was less potent in the \nproliferation assay  (EC50 = 4.9\xa0nmol/L) [133].\nBased on JBJ-07-149 (38), different linkers that bind \nthe piperazine group and connect the CRBN ligand \nwere evaluated. The compound with 3-PEG as the linker \n(DDC-01-163, 39) showed the strongest antiproliferative activity for EGFR L858R/T790M (Fig.\xa0 8). DDC-01-\n163 (39) induced the selective degradation mutant \nEGFR and inhibited the proliferation of cells expressing \nmutant EGFR in a dose- and time-dependent manner. \nDDC-01-163 (39) showed no activity in WT-EGFR Ba/\nF3 cells  (EC50 > 10\xa0 μmol/L) but inhibited the prolifera -\ntion of EGFR L858R/T790M Ba/F3 cells, including those \nexpressing EGFR L858R/T790M  (EC50 = 0.096\xa0 μmol/L), \nEGFR L858R/T790M/C797S  (EC50 = 0.041\xa0 μmol/L) and \nEGFR L858R/T790M/L718Q  (EC50 = 0.028\xa0 μmol/L). \nThe results in H1975 cells were consistent with those in \nBa/F3 cells. Osimertinib-resistant cell lines treated with \n0.1\xa0 μmol/L DDC-01-163 (39) showed EGFR L858R/\nT790M/C797S and EGFR L858R/T790M/L718Q degra -\ndation rates of 74% and 71%, respectively.\nJang et\xa0al. also identified the 2-hydroxy-5-fluorophenyl \nallosteric inhibitor JBJ-04-125-02 (6), which can be used \nFig. 8 Chemical structures and structure–activity relationships of an EGFR-targeted PROTAC: the rational design of DDC-01-163 (39)Page 21 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nas a single agent to inhibit the proliferation of Ba/F3 cells. \nFollowing the same strategy as that used to develop DDC-\n01-163 (39), this group designed JBJ-04-125-02 (6) as a \nPROTAC molecule and synthesized the allosteric EGFR \ndegrader JBJ-07–038 (40)  (EC50 = 0.48\xa0μmol/L). In addi -\ntion, JBJ-07-200 (41)  (EC50 = 0.15\xa0μmol/L) was obtained \nby replacing the hydroxyl group of JBJ-04-125-02 (6) with \nfluorine (Fig.\xa0 9), which could potentially improve mem -\nbrane permeability [133]. It is highly anticipated that \nthe further characteristic optimization and development \nof allosteric EGFR PROTACs will produce a valuable therapeutic strategy that will benefit more patients with \nEGFR-mutant disease.\nAccording to the first report by Zhao et\xa0al., EGFR deg -\nradation induced by PROTACs may be related to the \nautophagy pathway [134]. Qu et\xa0 al. [135] demonstrated \nfor the first time that in addition to the well-known ubiq -\nuitin/proteasome pathway, the ubiquitin/autophagy/lyso -\nsomal pathway participates in PROTAC-induced EGFR \ndegradation. Based on the EGFR inhibitor canertinib \n(41) and the CRBN ligand pomalidomide (an E3 ubiqui -\ntin ligase ligand), researchers generated two novel EGFR \nFig. 9 Chemical structures and EGFR inhibitory activities of EGFR-targeted PROTACs: the rational design of JBJ-07-200 (41)Page 22 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nPROTACs (Fig.\xa0 10), namely SIAIS125 (42) and SIAIS126 \n(43). These two EGFR degraders showed effective and \nselective antitumor activity in EGFR-TKI-resistant lung \ncancer cells.Dual PROTACs\nThe basic goal of modern drug discovery is to develop \nefficient and selective drugs for specific targets. How -\never, complex diseases such as cancer usually result from \ninteractions among multiple factors, synergistic effects \nof multiple disease-modifying factors, the upregulation \nof multiple receptors, and crosstalk between signaling \nFig. 10 Chemical structures and EGFR inhibitory activities of novel EGFR PROTACs (through the ubiquitin/autophagy/lysosomal degradation \nsystem): the rational design of SIAIS125 (42) and SIAIS126 (43)Page 23 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nnetworks. Tumor cells readily gain drug resistance by \nupregulating an alternative factor or transforming the \nsignaling pathway that promotes proliferation; therefore, \ntreatment focused on only a single target has limitations. \nIn addition to its issues related to drug resistance, single-\ntarget drugs also show reduced efficacy and can decrease \nthe quality of life of patients due to side effects and tissue \ntoxicity.\nTo overcome the deficiencies of single-target drugs, \nsingle hybrid molecules fused to two or more pharma -\ncophores have been designed to simultaneously tar -\nget two or more antitumor epitopes or targets. These \nhybrid molecules can simultaneously modulate multi -\nple targets or pathways and thus generally have better efficacy with fewer side effects. Based on this informa -\ntion and inspired by the great success of dual-targeted \ndrugs, especially dual-specific antibodies, Professor \nLi et\xa0al. combined the concepts of PROTACs and dual \ntargeting; this group used trifunctional natural amino \nacids as starlike core linkers to connect two inde -\npendent inhibitors, gefitinib (44) and olaparib (45), \nthat are linked to CRBN or VHL E3 ligands. The syn -\nthesized novel dual PROTACs can successfully and \nsimultaneously degrade EGFR and poly(ADP-ribose) \npolymerase (PARP) in cancer cells [136]. Among the \ndeveloped compounds, compound DP-V-4 (46) exhib -\nited the best ability to degrade EGFR and PARP in a \ndose- and time-dependent manner in H1299 cells and \nFig. 11 Chemical structure and antiproliferative activity of the dual PROTAC DP-V-4 (46) (through the ubiquitin–proteasome system)Page 24 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nhuman epidermal carcinoma A431 cells (Fig.\xa0 11). As \nthe first successful example of a dual PROTAC, this \nresearch will inject new vitality into the field of com -\nbination therapy for cancer. Moreover, these findings \nwill broaden the potential applications of the PROTAC \nmethod, open new fields of drug discovery, and over -\ncome the limitations of single-target therapy against \nEGFR.\nAnother new technology is the autophagy-targeting \nchimera (AUTAC), a small molecule that targets protein \ndegradation through autophagy and contains both a deg -\nradation tag (guanine derivative) and a warhead to pro -\nvide target specificity; AUTACs have a wider substrate \npanel than the ubiquitin–proteasome system [137–139]. \nTherefore, there is considerable potential for the design \nand development of AUTAC molecules to degrade EGFR.\nMonoclonal antibodies and\xa0ADCs\nFor patients with EGFR-mutant disease, there are tar -\ngeted therapies for tumors harboring EGFR-TKI-sensi -\ntizing mutations [140]. The EGFR monoclonal antibody \ncan bind to the extracellular domain of EGFR to compete \nwith EGF binding, thereby blocking downstream signal -\ning. The variable fragment (Fv) is composed of parts of \nthe light chain and heavy chain of the antibody and has \nunique antigen recognition function. The constant region \n(Fc) mediates innate immunity related to monoclonal \nantibodies, mainly by binding immune factors or cells to exert antitumor effects. These properties make antibod -\nies a favorable approach in targeted therapy, especially in \ncombination with other strategies. In addition, the inter -\nnalization and degradation of EGFR monoclonal anti -\nbody and receptor complexes can downregulate EGFR on \nthe surface of cancer cells. EGFR monoclonal antibodies \nare now standard-of-care therapies for head and neck \ncancer and colorectal cancer. Common EGFR monoclo -\nnal antibodies include cetuximab, necitumumab, panitu -\nmumab, matuzumab, and nimotuzumab. Antibody–drug \nconjugates (ADCs) are composed of three moieties: the \nantibody, linker, and drug (especially those with poten -\ntial cytotoxicity) (Fig.\xa0 12). Antibodies are equivalent to \nprecise arrows, and highly active cytotoxic drugs (the \npayload) correspond to the gunpowder on the arrows; \nthese drugs mainly include tubulin inhibitors (monome -\nthyl auristatin E, monomethyl auristatin F, mertansine, \nand ravtansine) and DNA-damaging agents (those that \ncause DNA double-strand breaks, DNA alkylation, DNA \nintercalation, and DNA cross-linking). It is difficult to \neffectively kill tumor cells with only cytotoxic drugs, but \nmonoclonal antibodies alone are too inefficient. ADCs \ncomposed of both the cytotoxic drug and a monoclonal \nantibody represent a more powerful combination. ADCs \ncan precisely target tumor cells by combining highly spe -\ncific monoclonal antibodies with highly toxic cytotoxic \ndrugs, thereby achieving a precise attack on EGFR-TKI-\nresistant cancer cells and filling the gap between antibody \nFig. 12 The mechanism of ADCsPage 25 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \ndrugs and traditional chemotherapy drugs. The ADC \napproach can improve both the drug specificity and the \ntreatment window. Being precise and efficient, ADCs \nhave therapeutic potential across cancer types and can \nalso induce tumor cell death via the bystander effect \n[141].\nHe et\xa0 al. developed a new ADC targeting EGFR, \nnamely SHR-A1307 (47) (Fig.\xa0 13), for the treatment of \nsolid tumors resistant or refractory to EGFR-targeted \ntherapy [142]. SHR-A1307 (47) has intermediate abil -\nity to block EGFR affinity for hR3 and selectively binds \nto cancer cells expressing EGFR while avoiding inhibi -\ntory effects on normal cells. In addition to increasing stability and reducing systemic toxicity, Fc domain \nengineering improved the pharmacokinetics. Although \nless frequent drug administration may reduce toxin \naccumulation, effective tumor cell killing with mini -\nmal toxicity were observed. In addition, SHR-A1307 \n(47) can effectively kill cancer cells that do not respond \nto current EGFR inhibitors and shows low nanomolar \nin\xa0vitro cytotoxicity in a broad spectrum of cancer cells \nwith different drug resistance mutations, thus provid -\ning an attractive treatment opportunity to overcome \nthe drug resistance of patients with EGFR-overexpress -\ning tumors.\nFig. 13 Chemical structure of ADCs targeting EGFR: SHR-A1307 (47) and MRG003 (48) (*Specific research data not disclosed)Page 26 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nTable 3 Combination therapy with the bypass pathway target\nTarget Representative compound* Structure Reference\nMET Cabozantinib\n49[145, 146]\nMET Crizotinib\n50[147]\nMET Savolitinib\n51[148]\nFGFR AZD4547\n52[149]\nALK Lorlatinib\n53[150]\nALK Brigatinib\n54[151]\nHER2 JQ1\n55[152]Page 27 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nMRG003 (48) [143, 144], the first EGFR ADC to enter \nthe clinical trials in China, is composed of a humanized \nanti-EGFR monoclonal antibody and the tubulin inhibi -\ntor MMAE coupled through a degradable VC (Val-Cit) \nlinker (Fig.\xa0 13). The phase I dose escalation and expan -\nsion study for patients with refractory solid tumors has \nbeen completed. Based on the results of the phase Ia \nand Ib clinical trials, Lepu Biosciences is currently con -\nducting phase II clinical trials of MRG003 monotherapy \nin China for recurrent or metastatic advanced head and \nneck squamous cell carcinoma, advanced NSCLC, biliary \ntract cancer, and nasopharyngeal carcinoma.\nCombination therapy strategy\nResistance to third-generation EGFR inhibitors medi -\nated by EGFR-independent mechanisms can develop \nthrough the activation of alternative bypass pathways and \nabnormal downstream signal transduction closely related \nto tumor growth, invasion and metastasis. In the clinic, \nHER2 mutation, high HGF expression, and abnormal \nactivation of MET, AXL, IGF1R and the FGFR pathway were found in patients with acquired resistance to third-\ngeneration EGFR-TKIs. Mutation or abnormal expres -\nsion of EGFR signaling pathway-related genes involved in \nthe Ras/Raf/MEK/ERK/MARK, PI3K/PDK1/Akt, PLC-γ \nand JAK/STAT pathways was also found. Importantly, \nthese aberrations can coexist in the same tumor and with \nEGFR-TKI tertiary mutations, which are the basis for \nthe complexity and heterogeneity of cancer evolution in \nresponse to EGFR-TKI treatment. Therefore, in combina -\ntion with third-generation EGFR-TKIs, targeting impor -\ntant components of alternative bypass pathways (Table\xa0 3) \n[145–154] and downstream signal transduction pathways \n(Table\xa0 4) [155–164] appears to be a promising treatment \nstrategy.\nMultitarget inhibitors\nCancer is a multifactorial disease, and single-target treat -\nments may have poor efficacy. As clinical targeted ther -\napy, EGFR kinase inhibitors are effective only when the \ncancer cells contain specific EGFR-activating mutations \nthat alter downstream signaling [165]. Moreover, only a *Osimertinib is a representative third-generation EGFR-TKITable 3 (continued)\nTarget Representative compound* Structure Reference\nHER2 Trastuzumab-DM1\n56[153]\nBRAF V600E Encorafenib (LGX818)\n57[64]\nAURK B PF-03814735\n58[60, 154]Page 28 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nTable 4 Combination therapy with targets in downstream signaling pathways\nTarget Representative compound* Structure Reference\nMEK Trametinib\n59[155, 156]\nMEK Selumetinib\n60[155–157]\nMEK PD0325901\n61[155]\nAKT Uprosertib (GSK2141795)\n62[158]\nAKT Capivasertib (AZD5363)\n63[158]\nAXL Cabozantinib\n64[159]\nAXL DS-1205b\n65[160]Page 29 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nsmall proportion of patients benefit from EGFR inhibitors \n[2]. In addition to activating mutations at the EGFR locus \nthat lead to drug resistance, a large number of genetic \nand epigenetic abnormalities may also lead to resistance \nto third-generation EGFR-TKIs. The emergence of intrin -\nsic and acquired resistance requires appropriate strate -\ngies to prevent serious side effects. Combination therapy \nhas additive or even synergistic effects, but due to vari -\nous dose-limiting toxicities and drug–drug interactions \ncaused by changes in pharmacokinetics, the simultane -\nous use of two or more drugs in the clinic is challeng -\ning. Therefore, as an alternative to combination therapy, \ndrugs targeting two or more objects have a lower risk of \ndrug–drug interactions and better pharmacokinetic and \nsafety profiles, which helps mitigate poor patient compli -\nance, off-target effects, and high development costs. Such \ntreatment regimens are more flexible and can represent \nan effective strategy for cancer therapy [166, 167]. The \neffectiveness of multitarget kinase inhibitors of WT and/\nor mutant EGFR has been extensively studied (Table\xa0 5) \n[59, 168–197]. Some EGFR-mutant cell lines are sensi -\ntive to multitarget inhibition and maintain certain levels \nof activity, highlighting the selectivity of multitarget com -\npounds and suggesting that multitarget inhibition can be used to circumvent acquired multidrug resistance to \nEGFR-targeted therapy without serious side effects.\nNatural products\nThe discovery of natural products offers new scaffolds for \ndrug development. Natural products are an important \nsource of compounds to overcome resistance to third-\ngeneration TKIs and provide ample possibilities for new \ndrug discovery. Honokiol (HNK) (98) is a natural prod -\nuct purified from Magnolia  used as a human nutritional \nsupplement, with good tolerance and safety profiles. \nMany preclinical studies have shown that HNK (98) has \npotential antitumor activity against different types of \ncancer. Zang et\xa0al. proved that the decrease in Mcl-1 and \nthe increase in BIM are the key mechanisms by which \nosimertinib induces the apoptosis of NSCLC cells with \nEGFR-TKI-sensitive mutations. HNK (98) and its deriva -\ntive CAz-p (99) in combination with osimertinib effec -\ntively reduced the survival and induced the apoptosis of \nEGFR ex19del/C797S (trans) double-mutant PC-9/2\xa0 M \ncells and EGFR ex19del/T790M/C797S (cis) triple-\nmutant PC-9/3\xa0 M cells [198]. It is highly encouraging \nthat HNK (98) and its derivatives may overcome clinical \nresistance to third-generation TKIs.*Osimertinib is a representative third-generation EGFR-TKITable 4 (continued)\nTarget Representative compound* Structure Reference\nAXL Yuanhuadine (YD)\n66[161]\nAXL Bemcentinib (R428)\n67[162]\nACK1 (R)-9b\n68[163, 164]Page 30 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nTable 5 Multitarget inhibitors\nNumber Target Pharmacophores Structure Activity\n69 [168] EGFR/FGFR1 Pyrimidine-2,4-diamines EGFRL858R/T790M\nIC50 = 43.1 nmol/L\nEGFRWT\nIC50 = 1138.7 nmol/L\nFGFR1WT\nIC50 = 17.6 nmol/L\nH1975 cells\nIC50 = 336.3 nmol/L\n70 [169] EGFR/Src Pyrimidine-4-amines K562 cells\nIC50 = 220 nmol/L\nA549 cells\nIC50 = 250 nmol/L\nEGFR inhibition rate = 33.15% \n(10 µmol/L)\nSrc inhibition rate = 72.12% \n(1 µmol/L)\n71 [170] EGFR/HER4 3-Cyanoquizolines EGFRL858R\nIC50 = 419 nmol/L\nEGFRWT\nIC50 = 2.4 nmol/L\nHER4\nIC50 = 0.03 nmol/L\n72 [171] EGFR/COX2 1,3,4-Oxadiazole scaffold EGFR\nIC50 = 280 nmol/L\nCOX2\nIC50 = 170 nmol/L\nUO-31 cells\nIC50 = 5800 nmol/L\n73 [172] EGFR/BRAF Spirobenzo[h]chromene \nderivativesEGFR\nIC50 = 1200 nmol/L\nBRAF\nIC50 = 2600 nmol/L\nA549 cells\nIC50 = 1780 nmol/L\nMCF-7 cells\nIC50 = 4090 nmol/L\nHT-29 cells\nIC50 = 4450 nmol/LPage 31 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nTable 5 (continued)\nNumber Target Pharmacophores Structure Activity\n74 [173] EGFRT790M/ALK Pyrimidine-2,4-diamines ALK\nIC50 = 18 nmol/L\nEGFRWT\nIC50 = 151 nmol/L\nEGFRT790M\nIC50 = 2 nmol/L\nEGFRL858R/T790M\nIC50 = 4 nmol/L\nDFCI032 cells\nIC50 = 170 nmol/L\nDFCI076 cells\nIC50 = 820 nmol/L\n75 [174] EGFRWT and \nmutant EGFR/\nALKPyrimidine-2,4-diamines EGFRWT\nIC50 = 108 nmol/L\nEGFRT790M\nIC50 = 3.9 nmol/L\nEGFRL858R/T790M\nIC50 = 3.6 nmol/L\nALKWT\nIC50 = 9.8 nmol/L\nALKR1275Q\nIC50 = 0.82 nmol/L\nALKL1196M\nIC50 = 0.59 nmol/L\nALKF1174L\nIC50 = 0.92 nmol/L\nALKC1156Y\nIC50 = 1.0 nmol/L\nH1975 cells\nGI50 = 15 nmol/L\nH3112 cells\nGI50 < 0.3 nmol/L\n76 [175] EGFR/ATX Pyrimidine-4-amines EGFR\nIC50 = 24.2 nmol/L\nATX\nIC50 = 29.1 nmol/L\nA549 cells\nIC50 = 4960 nmol/L\nMKN-45 cells\nIC50 = 3430 nmol/L\nSGC cells\nIC50 = 2910 nmol/L\nCFs cells\nIC50 = 1490 nmol/L\n77 [176] EGFR/AURK A Pyrimidine-4-amines AURK A\nIC50 = 1990 nmol/L\nEGFR\nIC50 = 3.76 nmol/L\n78 [177] EGFR/IGF1R Pyrimidine-2-amines EGFR\nIC50 = 35.5 nmol/L\nEGFRT790M\nIC50 = 66.0 nmol/L\nIGF1R\nIC50 = 52.0 nmol/LPage 32 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nTable 5 (continued)\nNumber Target Pharmacophores Structure Activity\n79 [178] EGFR/tubulin Pyrimidine-4-amines EGFR\nIC50 = 30 nmol/L\nTubulin assembly\nIC50 = 710 nmol/L\nHeLa cells\nIC50 = 1 nmol/L\nHT-29 cells\nIC50 = 20 nmol/L\nJurkat cells\nIC50 = 1 nmol/L\nRS4;11 cells\nIC50 = 1 nmol/L\n80 [179] EGFR/tubulin Chalcones EGFR\nIC50 = 39 nmol/L\nTubulin polymerization\nIC50 = 8840 nmol/L\nMCF-7 cells\nIC50 = 1650 nmol/L\nHCT-116 cells\nIC50 = 3610 nmol/L\n81 [180] EGFR/AKT Chalcones A549 cells\nIC50 = 3820 nmol/L\nMDA-MB-231 cells\nIC50 = 5890 nmol/L\nSKBR3 cells\nIC50 = 4790 nmol/L\n82 [181] EGFR/HDACs Pyrimidine-2-amines EGFRWT\nIC50 = 5700 nmol/L\nEGFRT790M\nIC50 = 5000 nmol/L\nHDACs\nIC50 = 85 nmol/L\nA549 cells\nIC50 = 2190 nmol/L\nHeLa cells\nIC50 = 1850 nmol/L\nMDA-MB-231 cells\nIC50 = 600 nmol/L\nMDA-MB-468 cells\nIC50 = 230 nmol/L\n83 [182] EGFR/PDGFR-β Pyrimidine-2,4-diamines EGFR Ki\nIC50 = 170 nmol/L\nPDGFR-β\nIC50 = 81 nmol/L\n84 [183] EGFR/NF-κB Quinazolines EGFR\nIC50 = 60.1 nmol/L\nNF-κB\nIC50 = 300 nmol/LPage 33 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nTable 5 (continued)\nNumber Target Pharmacophores Structure Activity\n85 [184] EGFR/c-Met 1,2,4-Oxadiazole derivate MDA-MB-231 cells\nIC50 = 200 nmol/L\nA459 cells  IC50 = 200 nmol/L\nPC9 cells  IC50 = 500 nmol/L\nH1975 cells  IC50 = 300 nmol/L\nCL68 cells  IC50 = 400 nmol/L\nCL97 cells\nIC50 = 500 nmol/L\n86 [185] EGFRT790M/c-Met Pyrimidine-2-amines EGFRT790M\nIC50 = 97 nmol/L\nc-Met\nIC50 = 518 nmol/L\n87 [186] EGFR/VEGFR-2 Pyrimidine-2,4-diamines EGFR Ki\nIC50 = 80 nmol/L\nVEGFR-2 Ki\nIC50 = 3240 nmol/L\nNCI-H460 cells GI = 25% \n(10 µmol/L)\n88 [187] EGFR/VEGFR-2 Quinazolines EGFR\nIC50 = 1.0 nmol/L\nVEGFR-2\nIC50 = 79.0 nmol/L\nHT-29 cells\nIC50 = 1760 nmol/L\nMCF7 cells\nIC50 = 7280 nmol/L\n89 [188] EGFR/VEGFR-2 Quinazolines EGFR\nIC50 = 0.69 nmol/L\nVEGFR-2\nIC50 = 67.84 nmol/L\n90 [59] EGFR/VEGFR-2 Quinazolines EGFR\nIC50 = 2.0 nmol/L\nVEGFR-2\nIC50 = 103.0 nmol/L\nA431 cells\nIC50 = 14.0 nmol/L\nH1975 cells\nIC50 = 130.0 nmol/LPage 34 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nTable 5 (continued)\nNumber Target Pharmacophores Structure Activity\n91 [189] EGFR/VEGFR-2 Quinazolines EGFR\nIC50 = 20 nmol/L\nVEGFR-2\nIC50 = 50 nmol/L\n92 [190] EGFR/HER2 Quinazolines EGFR\nIC50 = 0.69 nmol/L\nHER2\nIC50 = 42.1 nmol/L\nNCI-H1975 cells\nIC50 = 12.20 nmol/L\nHCC827 cells\nIC50 = 0.31 nmol/L\nA431 cells\nIC50 = 1.52 nmol/L\nMDA-MB-453 cells\nIC50 = 0.62 nmol/L\n93 [191] EGFR/HER2 Pyrimidine-4-amines EGFR\nIC50 = 186 nmol/L\nVEGFR-2\nIC50 = 254 nmol/L\n94 [192] EGFR/HER2 3-Cyanoquizolines EGFR\nIC50 = 597 nmol/L\nIGF1R\nIC50 = 908 nmol/L\nA431 cells\nIC50 = 1890 nmol/L\nSKBR3 cells\nIC50 = 1930 nmol/L\n95 [193–195] EGFR/HER2 Pyrimidinones EGFR\nIC50 = 60 nmol/L\nHER2\nIC50 = 300 nmol/L\nA549 cells\nIC50 = 280 nmol/L\n96 [196] EGFR/CSK Chalcones EGFR\nIC50 = 11,120 nmol/L\nCSK\nIC50 = 5160 nmol/LPage 35 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nOverexpression of MCL-1 induces acquired resistance \nto osimertinib. Combination therapy with MCL-1 inhibi -\ntors and osimertinib is a potential strategy to overcome \nresistance. Bufalin (100) is a natural product that belongs \nto the class of bufadienolide analogs. A recent study \nfound that bufalin (100) can reverse acquired resistance \nto osimertinib by inducing Ku70-mediated Mcl-1 degra -\ndation. Moreover, combined treatment with bufalin (100) \nand osimertinib triggered significant cell apoptosis and \nincreased the levels of cleaved caspase-3 and PARP [199].Wighteone (101) is a natural flavonoid compound \nwidely found in plants. Sun et\xa0al. reported that wighteone \n(101) docks at the ATP binding site of EGFR L858R/\nT790M and forms two hydrogen bonds with the carbonyl \ngroup of Gln791 and the amino group of Met793, indi -\ncating that it may directly bind to EGFR L858R/T790M. \nWighteone has a significant inhibitory effect on Ba/F3 \nand NCI-H1975 cells expressing EGFR L858R/T790M, \nwith  IC50 values of 1.88\xa0μmol/L and 5.70\xa0μmol/L, respec -\ntively [200] (Fig.\xa014).Table 5 (continued)\nNumber Target Pharmacophores Structure Activity\n97 [197] EGFR/CAIX Quinazolines EGFRWT\nIC50 = 27.0 nmol/L\nEGFRT790M\nIC50 = 9.2 nmol/L\nhCAII\nIC50 = 278.2 nmol/L\nhCAIX\nIC50 = 115.0 nmol/L\nA549 cells (hypoxia)\nIC50 = 2210 nmol/L\nA549 cells (normoxia)\nIC50 = 6450 nmol/L\nHeLa cells\nIC50 = 1850 nmol/L\nH1975 cells (hypoxia)\nIC50 = 1050 nmol/L\nH1975 cells (normoxia)\nIC50 = 1940 nmol/L\nFig. 14 Chemical structures of natural products with their synthetic analogs and other inhibitors (no specific  IC50 values are shown due to \nvariations in EGFR-mutant cell line survival and apoptosis assays)Page 36 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nOther strategies\nEGFR degradation based on\xa0the\xa0FBXL2‑Grp94‑EGFR axis\nXiao’s research group found that the F-box protein Fbxl2 \n(an E3 ubiquitin ligase) can target EGFR and EGFR-TKI-\nresistant mutants for proteasome-mediated degradation \nindependent of EGF stimulation. They also discovered \nthat glucose regulatory protein 94 (Grp94) protects \nEGFR from degradation by blocking the binding of Fbxl2 \nto EGFR. Through virtual screening of the DrugBank \ndatabase, small compounds that can bind to the Fbxo3-\napag domain were scored. Nebivolol (102) can be placed \nin the dumbbell-shaped cavity of the APAG region. \nThere are 5 amino acid residues in the center of this cav -\nity (I331, E341, T367, T368 and F369); T367 and T368 \nproject into the cavities of complementary shapes, form -\ning hydrophobic interactions with the ligand. The bind -\ning affinity of the Fbxo3 protein for endogenous Fbxl2 is \ngreatly reduced when these five amino acids are mutated \nindividually or in combination. Data suggest the potential \nof nebivolol (102) as a small molecule that can disrupt \nthe Fbxo3–Fbxl2 interaction. Increasing Fbxl2 levels with \nnebivolol (102) (Fig.\xa0 14) in combination with osimertinib \nor a Grp94 inhibitor (ganetespib) to target the FBXL2-\nGrp94-EGFR axis and thus destabilize EGFR is a possible \ntherapeutic strategy to overcome resistance to third-gen -\neration EGFR-TKIs [201].\nAKR1B1 inhibitors\nZhang et\xa0al. discovered that aldehyde ketone reductase \nfamily 1 member B1 (AKR1B1) interacts with STAT3 \nand activates the cystine transporter solute carrier \nfamily 7 member 11 (SLC7A11), which in turn leads \nto enhanced cystine uptake, glutathione synthesis flux, \nclearance of reactive oxygen species (ROS), protection \nagainst cell death, and EGFR-TKI resistance. The use of \nselective inhibitors (including the clinically approved \nanti-diabetic drug epalrestat) to inhibit AKR1B1 can \nrestore the sensitivity of drug-resistant cell lines to \nEGFR-TKIs and delay drug resistance in mice har -\nboring xenografted tumors derived from lung cancer \npatients [202].\nPGAM1 inhibitors\nPhosphoglycerate mutase 1 (PGAM1) is an important \nenzyme in the glycolysis pathway and is related to tumor \ncell metastasis [203]. HKB99 (103) (Fig.\xa0 14) is an allos -\nteric inhibitor of PGAM1 that significantly inhibits the \ngrowth and metastasis of NSCLC by affecting the meta -\nbolic activity and nonmetabolic functions of PGAM1 \n[204]. The docking model of the PGAM1-HKB99 com -\nplex shows that HKB99 (103) binds to the allosteric site \nof the adjacent substrate-binding pocket of PGAM1, thereby inhibiting the conversion of 3-PG to 2-PG and \nsignificantly reducing the metabolic activity of PGAM1. \nIn addition, HKB99 (103) can allosterically bind to \nPGAM1, weaken the interaction between PGAM1 \nand ACTA2, and inhibit the growth and metastasis of \nerlotinib-resistant lung cancer cells [205, 206]. There -\nfore, PGAM1 is a metabolic enzyme that may overcome \nEGFR-TKI resistance.\nNonoverlapping allosteric pockets—the X‑Pocket\nQiu et\xa0al. revealed the underlying mechanism of reverse \nallosteric communication in dual-targeted therapy. Allos -\nteric sites can be affected by orthomorphic drugs. The \nnonoverlapping allosteric pocket X-Pocket was discov -\nered in EGFR mutants; this pocket is mainly composed \nof nonconserved residues, including the hot spots K867, \nS895, and K960, that can cooperate with traditional TKIs \n[207]. It is a promising target for the design of selective \nconformationally restricted drugs, with great potential in \nterms of affinity, efficacy, and selectivity.\nDZ‑SIM inhibitors\nIn addition, researchers found that a group of near-infra -\nred heptamethine carbocyanine (DZ) fluorescent dyes, \nthe prototype of which is heptamethylamine carbocya -\nnine dye (IR-783) (104) (Fig.\xa0 14), have tumor-targeting \nactivity through differentially expressed organic anion \ntransport peptides on cancer cells [208]. This group of \norganic dyes can specifically deliver therapeutic pay -\nloads to tumor cells in the form of chemical conjugates. \nDZ-SIM was preliminarily synthesized; SIM specifi -\ncally targets 3-hydroxy-3-methylglutaryl-CoA reductase \n(HMGCR) in the endoplasmic reticulum. After specific \nuptake by tumor cells, DZ-SIM was enriched in subcel -\nlular organelles (including mitochondria and lysosomes). \nNSCLC cells were killed by mitochondrial damage, which \nmainly led to cytochrome C release into the cytoplasm, \nthereby activating the caspase-3-dependent apoptosis \ncascade. DZ-SIM inhibited the formation of cancer cell \ncolonies resistant to first-generation (H1650 and H1975) \nand third-generation EGFR-TKIs (PC9AR), and most \n IC50 values were lower than 10\xa0 μmol/L. DZ-SIM repre -\nsents a promising new therapy to overcome drug resist -\nance in patients with EGFR-mutant disease.\nSelection of\xa0individualized combination therapy\nFor patients who experience SCLC transformation, \nchemotherapy after the development of osimertinib \nresistance is an option. Research has shown that patients \nwith transformation to SCLC have higher response rates \nto etoposide, cisplatin, and paclitaxel. For patients with Page 37 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nunclear resistance mechanisms, chemotherapy is still \na treatment option. If the patient is asymptomatic or \nhas symptomatic local progression, osimertinib can be \ncombined with local treatment according to National \nComprehensive Cancer Network (NCCN) guidelines. \nCarboplatin, paclitaxel, bevacizumab, and atezolizumab \n(anti-PD-L1 antibody) are also options for patients \nwho experience systemic progression after osimertinib \ntreatment [209]. Whether chemotherapy can delay the \ndevelopment of resistance to third-generation EGFR-\nTKIs remains unknown. A study on osimertinib with or \nwithout chemotherapy as first-line therapy for patients \nwith EGFR-mutant NSCLC is currently recruiting \n(NCT04035486) [210].\nFor most patients, the PD-1/PD-L1 pathway is not the \nsole rate-limiting factor for antitumor immunity, and \nblocking the PD-1/PD-L1 axis is insufficient to activate \nan effective antitumor immune response [211]. Strate -\ngies that lead to acquired EGFR-TKI resistance, such as \nHGF, MET amplification, and EGFR T790M, also pro -\nmote immune escape in lung cancer by upregulating \nthe expression of PD-L1. Many combination strategies, \nincluding α-PD-1/PD-L1 plus chemotherapy, radio -\ntherapy, angiogenesis inhibitors, targeted therapy, other \nICIs, agonists of the costimulatory molecule, stimula -\ntor of interferon gene agonists, epigenetic modulators, \nor metabolic modulators, have been confirmed to have \nsuperior antitumor efficacy and a higher response rate. \nThe immunomodulatory effect of chemotherapy sug -\ngests that it might be a suitable partner for combina -\ntion with α-PD-1/PD-L1 to achieve rapid and long-term \ncancer control. During the KEYNOTE series of clini -\ncal trials (such as KEYNOTE-021, KEYNOTE-189, and \nKEYNOTE-407), pembrolizumab combined with stand -\nard chemotherapy led to better overall survival (OS) and \nprogression-free survival (PFS) in NSCLC patients and \nhas been approved by the FDA as first-line treatment for \nadvanced nonsquamous NSCLC [212, 213]. In addition, \nthe National Medical Products Administration (NMPA) \napproved sintilimab plus gemcitabine and platinum as \nfirst-line treatment for advanced squamous NSCLC \nbased on the results of ORIENT-12 [214]. In addition \nto α-PD-1-based approaches, α-PD-L1-based chemoim -\nmunotherapy has also attracted intense attention. The \nIMpower150 trial was the pioneer of this series of stud -\nies, and the FDA-approved atezolizumab plus bevaci -\nzumab, paclitaxel, and carboplatin as first-line treatment \nfor advanced nonsquamous NSCLC [215]. Subsequently, \nthe FDA-approved atezolizumab plus nab-paclitaxel and \ncarboplatin for nonsquamous NSCLC (based on the \nresults of IMpower130). Radiotherapy can also induce \nimmunogenic cell death and enhance the antitumor \nimmune response. The results of a phase 1 study showed that α-PD-1/PD-L1 plus chemoradiotherapy was toler -\nable in advanced NSCLC (NCT02621398), with promis -\ning clinical outcomes. In multiple clinical studies, such \nas IMpower150, angiogenesis inhibitors enhanced the \nefficacy of α-PD-1/PD-L1 [216]. Moreover, dual immune \ncheckpoint blockade or costimulatory molecule agonists \nplus α-PD-1/PD-L1 are also promising strategies. To date, \nthe FDA has approved ipilimumab plus nivolumab for \nNSCLC and melanoma, among others. Agonists target -\ning costimulatory pathways such as CD27/CD70, CD40/\nCD40L, and 4-1BB/4-1BBL could also enhance T-cell \nactivity and restore the antitumor immune response. \nHowever, bispecific/bifunctional antibodies simulta -\nneously block two molecules and thus have a strategic \nadvantage over combination therapy. For example, in the \nphase 1 NCT03710265 trial, SHR-1701 (TGF-β × PD-L1 \nbifunctional antibody) showed encouraging antitumor \nactivity [217].\nGiven the heterogeneity of mutations across patients, \nthe selection of individualized combination treatment \nstrategies could improve outcomes and mitigate treat -\nment resistance.\nDiscussion and\xa0future perspectives\nEGFR is an important target on tumor cells that pro -\nmotes mitosis and transformation. It is overexpressed in \nmany diseases and is particularly related to the occur -\nrence and development of cancer [3, 7, 8]. Tumors often \nhave prominent genomic and transcriptional hetero -\ngeneity that is closely related to EGFR-TKI resistance \n[40, 218]. Studies have shown that drug resistance can \ndevelop through EGFR-dependent and EGFR-independ -\nent mechanisms [24, 219]. The emergence of resistance to \nthird-generation EGFR-TKIs limits the clinical benefits \nfor patients, thus necessitating the further development \nof more effective strategies.\nTo date, fourth-generation EGFR-TKIs show promi -\nnent antitumor activity. Recent research has shown that \nfourth-generation inhibitors targeting allosteric sites \nand ATP-competitive sites of EGFR can achieve remark -\nable results against EGFR L858R/T790M and C797S. In \naddition to fourth-generation EGFR-TKIs, combination \ntreatments, monoclonal antibodies, and bispecific anti -\nbodies are significantly contributing to the treatment of \ntumors harboring the C797S mutation. While the C797S \nmutation is only one of the numerous drug resistance \nmechanisms, it is necessary to overcome other muta -\ntions by designing and developing new noncovalent \nATP-competitive inhibitors that form hydrogen bonds \nwith mutated residues in the EGFR ATP pocket (such \nas Lys745 and Asn842). The rational design of selective \nEGFR inhibitors that bind to both the ATP and allosteric \nsites of the EGFR kinase domain, that is, adding allosteric Page 38 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \ninhibitor elements to the compound skeleton at the ATP \nbinding site, will help optimize and improve the mutation \nselectivity of compounds and lead to the identification \nof small molecules with good kinase inhibitory activity. \nHowever, the cellular activity of such compounds needs \nto be further improved, and future research directions \nshould focus on the structural optimization of current \nlead compounds to obtain EGFR inhibitors with better \nmutation selectivity. Targeted protein degradation tech -\nnology provides a new research direction for overcoming \nresistance to third-generation EGFR inhibitors. Consid -\nering the significance of overcoming allosteric hindrance \nby triple-mutant EGFR, allosteric EGFR degraders were \ndeveloped. In addition, dual PROTACs have emerged in \nthe field of cancer combination therapy; dual PROTACs \ncan be designed with two targets, such as tumor immune \ntargets plus adjuvant immune targets or energy metabo -\nlism targets and epigenetic targets plus antiapoptotic tar -\ngets, to further overcome resistance of third-generation \nEGFR inhibitors and provide a better curative effect. Of \ncourse, the larger molecular weight of dual PROTACs \nwill affect their druggability and pharmacokinetics, \nbut perhaps nanodrug delivery systems can be utilized \nto improve drug absorption or optimized by simplify -\ning the inhibitor moiety and maintaining the minimum \npharmacophore. In addition, ADCs containing a small-\nmolecule cytotoxic compound and a monoclonal anti -\nbody targeting a cancer target have attracted attention. \nThe ADC MRG003 has entered clinical trials with great \ndevelopment and application prospects. The activation of \nalternative pathways and histological transformation are \nimportant mechanisms of resistance to third-generation \nEGFR inhibitors. The combined use of third-generation \ninhibitors and related pathway blockers is another impor -\ntant approach. To prevent the toxicity and side effects of \nmultidrug combinations, drugs with multiple pharmaco -\nlogical activities were developed and proven to have more \nadvantages than combination therapy. Multitarget kinase \ndrugs have become a favorable choice due to their attrac -\ntive pharmacokinetic characteristics and safety profiles. \nNatural compounds have received much research atten -\ntion due to their potential antitumor effects. Based on the \nmolecular mechanism of inhibition, natural compounds \ncan be modified to provide new insights for effectively \novercoming resistance to third-generation EGFR-TKIs. \nLast but not least, the discovery of EGFR degraders based \non the FBXL2-Grp94-EGFR axis, AKR1B1 and PGAM1 \ninhibitors, DZ-SIM, and the nonoverlapping allosteric \npocket X-Pocket provides promising support for the fur -\nther development of strategies to overcome resistance \nto third-generation EGFR inhibitors. The mechanism of \nresistance to third-generation EGFR-TKIs is very com -\nplex, is impacted by EGFR mutations, and differs among patients and tumor sites. Thus, next-generation sequenc -\ning (NGS) of blood-based circulating tumor DNA \n(ctDNA) or tissue samples to elucidate the resistance \nmechanism will be valuable for guiding future therapeu -\ntic approaches and for clinical research on novel combi -\nnation therapies to overcome drug resistance. Moreover, \nindividualized combination treatment strategies could \nalso improve treatment efficacy and mitigate treatment \nresistance.\nEGFR is a verified target for antitumor therapy in a \nbroad spectrum of cancers. Facilitated by versatile strate -\ngies in the field of medicinal chemistry, better approaches \nare anticipated for overcoming the hurdle of drug resist -\nance to provide new hope for patients.\nConclusion\nAs a crucial “controller” that is related to the inhibi -\ntion of tumor cell proliferation, angiogenesis, invasion, \nmetastasis, and apoptosis, EGFR actively participates in \nmalignant disease progression. However, the intrinsic \nand acquired resistance in primary and recurrent can -\ncer which is mediated by EGFR mutations after target \ntreatment leads to difficult therapeutic. Understanding \nthe complex resistance mechanisms of EGFR-TKIs and \ndeveloping potential strategies to combat it could be of \npotential interest for improving the individual therapeu -\ntic strategies for cancer.\nAbbreviations\nACK1: Activated Cdc42-associated kinase 1; ADC: Antibody drug conjugates; \nAKR1B1: Aldehyde–ketone reductase family 1 member B1; A-loop: Activation \nloop; ATP: Adenosine triphosphate; AURK: Aurora kinases; AUTAC : Autophagy-\ntargeting chimera; Bcl-2: B cell lymphoma-2; BRAF: V-RAF murine sarcoma \nviral oncogene homolog B1; CCND: Cyclin D; CCNE1: Cyclin E1; CDK4/6: \nCyclin-dependent kinase 4/6; cIAP1: Cellular inhibitor of apoptosis protein 1; \nCRBN: Cereblon; DFG: Asp-Phe-Gly; DZ-SIM: DZ-SIMvastatin; EGF: Epidermal \ngrowth factor; EGFR: Epidermal growth factor receptor; EGFR-TKIs: Epidermal \ngrowth factor receptor tyrosine kinase inhibitors; EMT: Epithelial–mesen-\nchymal transformation; EMT-TFs: EMT-induced transcription factors; ERC1: \nExcision repair cross-complementation 1; FGF: Fibroblast growth factor; FGFR: \nFibroblast growth factor receptor; Fv: Variable region fragment; Grp94: Glucose \nregulatory protein 94; HB-EGF: EGF-like growth factor; HMGCR : 3-Hydroxy-\n3-methylglutaryl-CoA reductase; HNK: Honokiol; ICIs: Immune checkpoint \ninhibitors; IGF1R: Insulin-like growth factor receptor 1; LUAD: Lung adenocar -\ncinoma; MDM2: Mouse double minute 2; NCCN: National Comprehensive Can-\ncer Network; NSCLC: Non-small cell lung cancer; NTRK1: Neurotrophic tyrosine \nreceptor kinase 1; PARP: Poly(ADP-ribose) polymerase; PGAM1: Phosphoglycer -\nate mutase 1; PIK3CA: Phosphatidylinositol 4,5-bisphosphate 3-kinase catalytic \nsubunit; PROTAC : Proteolysis-targeting chimera; PTEN: Phosphatase and tensin \nhomolog; ROS: Reactive oxygen species; RTK: Receptor tyrosine kinase; SCLC: \nSmall cell lung cancer; SLC7A11: Solute carrier family 7 member 11; SPP1: \nSecreted phosphoprotein 1; STAT  : Signal sensor and transcription activator; \nTACC3: Transforming acid helix protein 3; TGF: Transforming growth factor; \nTMP3: Thrombopoietin mimetic peptide 3; VHL: Von Hippel–Lindau; WT: Wild-\ntype; WT-EGFR: Wild-type EGFR.\nAcknowledgements\nNot applicable.Page 39 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \nAuthor contributions\nWL, YW, and LO conceived the project, supervised the project, and revised the \nmanuscript. KS, GW, and JP summed up the literature, drafted the manuscript, \nand drew the figures. JP and JZ collected and organized the inhibitors. JW \nand GW proofread the structures and figures. All authors approved the final \nmanuscript.\nFunding\nThis work was supported by grants from the National Natural Science Founda-\ntion of China (Grants 82037718, 81922064, 22177083), the Fundamental \nResearch Funds for the Central Universities (SCU2022D025), 1.3.5 project for \ndisciplines of excellent, West China Hospital, Sichuan University (ZYJC18001), \nthe Sichuan Science and Technology Program (grant number 2022NSFSC1290, \n2019YFS0003), and West China Nursing Discipline Development Special Fund \nProject, Sichuan University (Grant HXHL21011).\nAvailability of data and materials\nThe material supporting the conclusion of this review has been included \nwithin the article.\nDeclarations\nEthics approval and consent to participate\nThis is not applicable for this review.\nConsent for publication\nThis is not applicable for this review.\nCompeting interests\nThe authors declare no competing financial interests or personal relationships \nthat could have appeared to influence the work reported in this paper.\nAuthor details\n1 Department of Respiratory and Critical Care Medicine, Targeted Tracer \nResearch and Development Laboratory, Institute of Respiratory Health, \nInnovation Center of Nursing Research, Nursing Key Laboratory of Sichuan \nProvince, West China Hospital, Sichuan University, Chengdu 610041, China. \n2 Department of Pharmaceutical Sciences, College of Pharmacy, University \nof Tennessee Health Science Center, Memphis, TN 38163, USA. 3 State Key \nLaboratory of Biotherapy and Cancer Center, West China Hospital, and Collab -\norative Innovation Center of Biotherapy, Sichuan University, Chengdu 610041, \nChina. 4 Precision Medicine Key Laboratory of Sichuan Province and Preci-\nsion Medicine Research Center, West China Hospital, Sichuan University, \nChengdu 610041, China. 5 Tianfu Jincheng Laboratory, Chengdu 610041, \nChina. \nReceived: 23 March 2022   Accepted: 28 June 2022\nReferences\n 1. Amelia T, Kartasasmita RE, Ohwada T, Tjahjono DH. Structural insight \nand development of EGFR tyrosine kinase inhibitors. Molecules. \n2022;27:819.\n 2. da Cunha SG, Shepherd FA, Tsao MS. EGFR mutations and lung cancer. \nAnnu Rev Pathol. 2011;6:49–69.\n 3. Campbell ID, Bork P . Epidermal growth factor-like modules. Curr Opin \nStruct Biol. 1993;3:385–92.\n 4. Roskoski R. The ERBB/HER family of protein-tyrosine kinases and cancer. \nPharmacol Res. 2014;79:34–74.\n 5. Gazdar AF. Activating and resistance mutations of EGFR in non-small-\ncell lung cancer: role in clinical response to EGFR tyrosine kinase inhibi-\ntors. Oncogene. 2009;28(Suppl 1):S24–31.\n 6. Herbst RS, Langer CJ. Epidermal growth factor receptors as a target for \ncancer treatment: the emerging role of IMC-C225 in the treatment of \nlung and head and neck cancers. Semin Oncol. 2002;29:27–36.\n 7. Normanno N, Bianco C, De Luca A, Salomon DS. The role of EGF-related \npeptides in tumor growth. Front Biosci. 2001;6:D685–707. 8. Thomas R, Weihua Z. Rethink of EGFR in cancer with its kinase inde -\npendent function on board. Front Oncol. 2019;9:800.\n 9. Sabbah DA, Hajjo R, Sweidan K. Review on epidermal growth factor \nreceptor (EGFR) structure, signaling pathways, interactions, and recent \nupdates of EGFR inhibitors. Curr Top Med Chem. 2020;20:815–34.\n 10. Cross DA, Ashton SE, Ghiorghiu S, Eberlein C, Nebhan CA, Spitzler \nPJ, et al. AZD9291, an irreversible EGFR TKI, overcomes T790M-\nmediated resistance to EGFR inhibitors in lung cancer. Cancer Discov. \n2014;4:1046–61.\n 11. Ricordel C, Friboulet L, Facchinetti F, Soria JC. Molecular mechanisms \nof acquired resistance to third-generation EGFR-TKIs in EGFR T790M-\nmutant lung cancer. Ann Oncol. 2018;29:i28–37.\n 12. Roskoski R Jr. Classification of small molecule protein kinase inhibitors \nbased upon the structures of their drug-enzyme complexes. Pharmacol \nRes. 2016;103:26–48.\n 13. Sequist LV, Yang JC, Yamamoto N, O’Byrne K, Hirsh V, Mok T, et al. \nPhase iii study of afatinib or cisplatin plus pemetrexed in patients with \nmetastatic lung adenocarcinoma with EGFR mutations. J Clin Oncol. \n2013;31:3327–34.\n 14. Wu YL, Zhou C, Hu CP , Feng J, Lu S, Huang Y, et al. Afatinib versus \ncisplatin plus gemcitabine for first-line treatment of Asian patients \nwith advanced non-small-cell lung cancer harbouring EGFR mutations \n(LUX-Lung 6): an open-label, randomised phase 3 trial. Lancet Oncol. \n2014;15:213–22.\n 15. Zhang H. Three generations of epidermal growth factor receptor tyros-\nine kinase inhibitors developed to revolutionize the therapy of lung \ncancer. Drug Des Devel Ther. 2016;10:3867–72.\n 16. Pao W, Chmielecki J. Rational, biologically based treatment of EGFR-\nmutant non-small-cell lung cancer. Nat Rev Cancer. 2010;10:760–74.\n 17. Jiang T, Zhou C. Clinical activity of the mutant-selective EGFR inhibitor \nAZD9291 in patients with EGFR inhibitor-resistant non-small cell lung \ncancer. Transl Lung Cancer Res. 2014;3:370–2.\n 18. Yan XE, Zhu SJ, Liang L, Zhao P , Choi HG, Yun CH. Structural basis of \nmutant-selectivity and drug-resistance related to CO-1686. Oncotarget. \n2017;8:53508–17.\n 19. Leonetti A, Sharma S, Minari R, Perego P , Giovannetti E, Tiseo M. Resist -\nance mechanisms to osimertinib in EGFR-mutated non-small cell lung \ncancer. Br J Cancer. 2019;121:725–37.\n 20. Hata AN, Niederst MJ, Archibald HL, Gomez-Caraballo M, Siddiqui FM, \nMulvey HE, et al. Tumor cells can follow distinct evolutionary paths to \nbecome resistant to epidermal growth factor receptor inhibition. Nat \nMed. 2016;22:262–9.\n 21. Guibert N, Barlesi F, Descourt R, Lena H, Besse B, Beau-Faller M, et al. \nCharacteristics and outcomes of patients with lung cancer harboring \nmultiple molecular alterations: results from the IFCT study biomarkers \nfrance. J Thorac Oncol. 2017;12:963–73.\n 22. Li X, Wang S, Li B, Wang Z, Shang S, Shao Y, et al. Bim deletion polymor -\nphism confers resistance to osimertinib in EGFR T790M lung cancer: a \ncase report and literature review. Target Oncol. 2018;13:517–23.\n 23. Eck MJ, Yun CH. Structural and mechanistic underpinnings of the differ -\nential drug sensitivity of EGFR mutations in non-small cell lung cancer. \nBiochim Biophys Acta. 2010;1804:559–66.\n 24. Westover D, Zugazagoitia J, Cho BC, Lovly CM, Paz-Ares L. Mechanisms \nof acquired resistance to first- and second-generation EGFR tyrosine \nkinase inhibitors. Ann Oncol. 2018;29:i10–9.\n 25. Zhou W, Ercan D, Chen L, Yun CH, Li D, Capelletti M, et al. Novel \nmutant-selective EGFR kinase inhibitors against EGFR T790M. Nature. \n2009;462:1070–4.\n 26. Zhang Q, Zhang XC, Yang JJ, Yang ZF, Bai Y, Su J, et al. EGFR L792H \nand G796R: two novel mutations mediating resistance to the third-\ngeneration EGFR tyrosine kinase inhibitor osimertinib. J Thorac Oncol. \n2018;13:1415–21.\n 27. Zheng D, Hu M, Bai Y, Zhu X, Lu X, Wu C, et al. EGFR G796D mutation \nmediates resistance to osimertinib. Oncotarget. 2017;8:49671–9.\n 28. Ou SI, Cui J, Schrock AB, Goldberg ME, Zhu VW, Albacker L, et al. \nEmergence of novel and dominant acquired EGFR solvent-front \nmutations at Gly796 (G796S/R) together with C797S/R and L792F/H \nmutations in one EGFR (L858R/T790M) NSCLC patient who pro -\ngressed on osimertinib. Lung Cancer. 2017;108:228–31.\n 29. Castellano GM, Aisner J, Burley SK, Vallat B, Yu HA, Pine SR, \net al. A novel acquired exon 20 EGFR M766Q mutation in lung Page 40 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \nadenocarcinoma mediates osimertinib resistance but is sensitive to \nneratinib and poziotinib. J Thorac Oncol. 2019;14:1982–8.\n 30. Liu J, Jin B, Su H, Qu X, Liu Y. Afatinib helped overcome subsequent \nresistance to osimertinib in a patient with NSCLC having leptomenin-\ngeal metastasis baring acquired EGFR L718Q mutation: a case report. \nBMC Cancer. 2019;19:702.\n 31. Bersanelli M, Minari R, Bordi P , Gnetti L, Bozzetti C, Squadrilli A, \net al. L718Q mutation as new mechanism of acquired resistance to \nAZD9291 in EGFR -mutated NSCLC. J Thorac Oncol. 2016;11:e121–3.\n 32. Callegari D, Ranaghan KE, Woods CJ, Minari R, Tiseo M, Mor M, et al. \nL718Q mutant EGFR escapes covalent inhibition by stabilizing a non-\nreactive conformation of the lung cancer drug osimertinib. Chem Sci. \n2018;9:2740–9.\n 33. Yang Z, Yang J, Chen Y, Shao YW, Wang X. Acquired EGFR L718V \nmutation as the mechanism for osimertinib resistance in a \nT790M-negative non-small-cell lung cancer patient. Target Oncol. \n2019;14:369–74.\n 34. Fassunke J, Muller F, Keul M, Michels S, Dammert MA, Schmitt A, et al. \nOvercoming EGFR (G724S)-mediated osimertinib resistance through \nunique binding characteristics of second-generation EGFR inhibitors. \nNat Commun. 2018;9:4655.\n 35. Tu HY, Ke EE, Yang JJ, Sun YL, Yan HH, Zheng MY, et al. A comprehen-\nsive review of uncommon EGFR mutations in patients with non-small \ncell lung cancer. Lung Cancer. 2017;114:96–102.\n 36. Xu J, Jin B, Chu T, Dong X, Yang H, Zhang Y, et al. EGFR tyrosine kinase \ninhibitor (TKI) in patients with advanced non-small cell lung cancer \n(NSCLC) harboring uncommon EGFR mutations: a real-world study in \nchina. Lung Cancer. 2016;96:87–92.\n 37. Shen YC, Tseng GC, Tu CY, Chen WC, Liao WC, Chen WC, et al. \nComparing the effects of afatinib with gefitinib or erlotinib in \npatients with advanced-stage lung adenocarcinoma harboring non-\nclassical epidermal growth factor receptor mutations. Lung Cancer. \n2017;110:56–62.\n 38. Ercan D, Choi HG, Yun CH, Capelletti M, Xie T, Eck MJ, et al. EGFR muta-\ntions and resistance to irreversible pyrimidine-based EGFR inhibitors. \nClin Cancer Res. 2015;21:3913–23.\n 39. Piotrowska Z, Isozaki H, Lennerz JK, Gainor JF, Lennes IT, Zhu VW, et al. \nLandscape of acquired resistance to osimertinib in EGFR -mutant \nNSCLC and clinical validation of combined EGFR and RET inhibition \nwith osimertinib and BLU-667 for acquired RET fusion. Cancer Discov. \n2018;8:1529–39.\n 40. Piotrowska Z, Niederst MJ, Karlovich CA, Wakelee HA, Neal JW, \nMino-Kenudson M, et al. Heterogeneity underlies the emergence of \nEGFRT790 wild-type clones following treatment of T790M-positive \ncancers with a third-generation EGFR inhibitor. Cancer Discov. \n2015;5:713–22.\n 41. Nukaga S, Yasuda H, Tsuchihara K, Hamamoto J, Masuzawa K, Kawada \nI, et al. Amplification of EGFR wild-type alleles in non-small cell lung \ncancer cells confers acquired resistance to mutation-selective EGFR \ntyrosine kinase inhibitors. Cancer Res. 2017;77:2078–89.\n 42. Huang C, Zou Q, Liu H, Qiu B, Li Q, Lin Y, et al. Management of non-small \ncell lung cancer patients with MET exon 14 skipping mutations. Curr \nTreat Opt Oncol. 2020;21:33.\n 43. Mueller KL, Madden JM, Zoratti GL, Kuperwasser C, List K, Boerner JL. \nFibroblast-secreted hepatocyte growth factor mediates epidermal \ngrowth factor receptor tyrosine kinase inhibitor resistance in triple-\nnegative breast cancers through paracrine activation of MET. Breast \nCancer Res. 2012;14:R104.\n 44. Hsu CC, Liao BC, Liao WY, Markovets A, Stetson D, Thress K, et al. Exon \n16-skipping HER2 as a novel mechanism of osimertinib resistance in \nEGFR L858R/T790M-positive non-small cell lung cancer. J Thorac Oncol. \n2020;15:50–61.\n 45. Ou S-HI, Madison R, Robichaux JP , Ross JS, Miller VA, Ali SM, et al. Char -\nacterization of 648 non-small cell lung cancer (NSCLC) cases with 28 \nunique HER2 exon 20 insertions. J Clin Oncol. 2019;37:9063–63.\n 46. Gao G, Li X, Wang Q, Zhang Y, Chen J, Shu Y, et al. Single-arm, phase \nii study of pyrotinib in advanced non-small cell lung cancer (NSCLC) \npatients with HER2 exon 20 mutation. J Clin Oncol. 2019;37:9089–189.\n 47. Wu SG, Shih JY. Management of acquired resistance to EGFR TKI-\ntargeted therapy in advanced non-small cell lung cancer. Mol Cancer. \n2018;17:38. 48. Zhu J, Yang Q, Xu W. Iterative upgrading of small molecular tyrosine \nkinase inhibitors for EGFR mutation in NSCLC: necessity and perspec-\ntive. Pharmaceutics. 2021;13:1500.\n 49. Taniguchi H, Yamada T, Wang R, Tanimura K, Adachi Y, Nishiyama A, \net al. AXL confers intrinsic resistance to osimertinib and advances the \nemergence of tolerant cells. Nat Commun. 2019;10:259.\n 50. Yano S, Yamada T, Takeuchi S, Tachibana K, Minami Y, Yatabe Y, et al. \nHepatocyte growth factor expression in EGFR mutant lung cancer \nwith intrinsic and acquired resistance to tyrosine kinase inhibitors in a \nJapanese cohort. J Thorac Oncol. 2011;6:2011–7.\n 51. Kim TM, Song A, Kim DW, Kim S, Ahn YO, Keam B, et al. Mechanisms of \nacquired resistance to AZD9291 a mutation-selective, irreversible EGFR \ninhibitor. J Thorac Oncol. 2015;10:1736–44.\n 52. Papadimitrakopoulou VA, Wu YL, Han JY, Ahn MJ, Ramalingam SS, John \nT, et al. Analysis of resistance mechanisms to osimertinib in patients \nwith EGFR T790M advanced NSCLC from the AURA3 study. Ann Oncol. \n2018;29:741–841.\n 53. Tanaka H, Sakagami H, Kaneko N, Konagai S, Yamamoto H, Matsuya T, \net al. Mutant-selective irreversible EGFR inhibitor, naquotinib, inhibits \ntumor growth in NSCLC models with EGFR-activating mutations, T790M \nmutation, and AXL overexpression. Mol Cancer Ther. 2019;18:1366–73.\n 54. Park JH, Choi YJ, Kim SY, Lee JE, Sung KJ, Park S, et al. Activation of the \nIGF1R pathway potentially mediates acquired resistance to mutant-\nselective 3rd-generation EGF receptor tyrosine kinase inhibitors in \nadvanced non-small cell lung cancer. Oncotarget. 2016;7:22005–15.\n 55. Carmena M, Earnshaw WC. The cellular geography of aurora kinases. \nNat Rev Mol Cell Biol. 2003;4:842–54.\n 56. Cheetham GMT, Knegtel RMA, Coll JT, Renwick SB, Swenson L, Weber \nP , et al. Crystal structure of aurora-2, an oncogenic serine/threonine \nkinase*. J Biol Chem. 2002;277:42419–22.\n 57. Pradhan T, Gupta O, Singh G, Monga V. Aurora kinase inhibitors as \npotential anticancer agents: recent advances. Eur J Med Chem. \n2021;221:113495.\n 58. Falchook GS, Bastida CC, Kurzrock R. Aurora kinase inhibitors in \noncology clinical trials: current state of the progress. Semin Oncol. \n2015;42:832–48.\n 59. Hu L, Fan M, Shi S, Song X, Wang F, He H, et al. Dual target inhibitors \nbased on EGFR: promising anticancer agents for the treatment of \ncancers (2017-). Eur J Med Chem. 2022;227:113963.\n 60. Tanaka K, Yu HA, Yang S, Han S, Selcuklu SD, Kim K, et al. Targeting \naurora b kinase prevents and overcomes resistance to EGFR inhibitors \nin lung cancer by enhancing BIM- and PUMA-mediated apoptosis. \nCancer Cell. 2021;39(1245–61):e6.\n 61. Mao C, Qiu LX, Liao RY, Du FB, Ding H, Yang WC, et al. KRAS mutations \nand resistance to EGFR-TKIs treatment in patients with non-small cell \nlung cancer: a meta-analysis of 22 studies. Lung Cancer. 2010;69:272–8.\n 62. Sunaga N, Shames DS, Girard L, Peyton M, Larsen JE, Imai H, et al. \nKnockdown of oncogenic KRAS in non-small cell lung cancers sup -\npresses tumor growth and sensitizes tumor cells to targeted therapy. \nMol Cancer Ther. 2011;10:336–46.\n 63. Leonetti A, Facchinetti F, Rossi G, Minari R, Conti A, Friboulet L, et al. Braf \nin non-small cell lung cancer (NSCLC): pickaxing another brick in the \nwall. Cancer Treat Rev. 2018;66:82–94.\n 64. Ho CC, Liao WY, Lin CA, Shih JY, Yu CJ, Yang JC. Acquired BRAF V600E \nmutation as resistant mechanism after treatment with osimertinib. J \nThorac Oncol. 2017;12:567–72.\n 65. Ramalingam SS, Cheng Y, Zhou C, Ohe Y, Imamura F, Cho BC, et al. \nMechanisms of acquired resistance to first-line osimertinib: preliminary \ndata from the phase iii flaura study. Ann Oncol. 2018;29:viii740.\n 66. Zhao M, Gao FH, Wang JY, Liu F, Yuan HH, Zhang WY, et al. JAK2/STAT3 \nsignaling pathway activation mediates tumor angiogenesis by upregu-\nlation of VEGF and bFGF in non-small-cell lung cancer. Lung Cancer. \n2011;73:366–74.\n 67. Chaib I, Karachaliou N, Pilotto S, Codony Servat J, Cai X, Li X, et al. \nCo-activation of STAT3 and YES-associated protein 1 (YAP1) pathway in \nEGFR-mutant NSCLC. J Natl Cancer Inst. 2017;109.\n 68. Soria J-C, Lee H-Y, Lee JI, Wang L, Issa J-P , Kemp BL, et al. Lack of PTEN \nexpression in non-small cell lung cancer could be related to promoter \nmethylation. Clin Cancer Res. 2002;8:1178–84.Page 41 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \n 69. Zhang T, Qu R, Chan S, Lai M, Tong L, Feng F, et al. Discovery of a novel \nthird-generation EGFR inhibitor and identification of a potential combi-\nnation strategy to overcome resistance. Mol Cancer. 2020;19:90.\n 70. Zhu L, Chen Z, Zang H, Fan S, Gu J, Zhang G, et al. Targeting c-Myc \nto overcome acquired resistance of EGFR mutant NSCLC cells to the \nthird-generation EGFR tyrosine kinase inhibitor, osimertinib. Cancer Res. \n2021;81:4822–34.\n 71. Weng CH, Chen LY, Lin YC, Shih JY, Lin YC, Tseng RY, et al. Epithelial–\nmesenchymal transition (emtEMT) beyond EGFR mutations per se is a \ncommon mechanism for acquired resistance to EGFR TKI. Oncogene. \n2019;38:455–68.\n 72. Brabletz T, Kalluri R, Nieto MA, Weinberg RA. EMT in cancer. Nat Rev \nCancer. 2018;18:128–34.\n 73. Kong W, Yang H, He L, Zhao JJ, Coppola D, Dalton WS, et al. Micro -\nRNA-155 is regulated by the transforming growth factor beta/Smad \npathway and contributes to epithelial cell plasticity by targeting RhoA. \nMol Cell Biol. 2008;28:6773–84.\n 74. Liu CH, Huang Q, Jin ZY, Zhu CL, Liu Z, Wang C. miR-21 and KLF4 jointly \naugment epithelialmesenchymal transition via the Akt/ERK1/2 path-\nway. Int J Oncol. 2017;50:1109–15.\n 75. Han Z, Zhou X, Li S, Qin Y, Chen Y, Liu H. Inhibition of miR-23a increases \nthe sensitivity of lung cancer stem cells to erlotinib through PTEN/PI3K/\nAkt pathway. Oncol Rep. 2017;38:3064–70.\n 76. Shen H, Zhu F, Liu J, Xu T, Pei D, Wang R, et al. Alteration in Mir-21/PTEN \nexpression modulates gefitinib resistance in non-small cell lung cancer. \nPLoS ONE. 2014;9:e103305.\n 77. Fardi M, Solali S, Farshdousti HM. Epigenetic mechanisms as a \nnew approach in cancer treatment: an updated review. Genes Dis. \n2018;5:304–11.\n 78. Del Re M, Arrigoni E, Restante G, Passaro A, Rofi E, Crucitta S, et al. \nConcise review: resistance to tyrosine kinase inhibitors in non-small cell \nlung cancer: the role of cancer stem cells. Stem Cells. 2018;36:633–40.\n 79. Papadimitrakopoulou VA, Wu YL, Han JY, Ahn MJ, Ramalingam SS, John \nT, et al. Analysis of resistance mechanisms to osimertinib in patients \nwith EGFR T790M advanced NSCLC from the AURA3 study. Ann Oncol. \n2018;29:viii741.\n 80. Dorantes-Heredia R, Ruiz-Morales JM, Cano-Garcia F. Histopathologi-\ncal transformation to small-cell lung carcinoma in non-small cell lung \ncarcinoma tumors. Transl Lung Cancer Res. 2016;5:401–12.\n 81. Sequist LV, Waltman BA, Dias-Santagata D, Digumarthy S, Turke AB, \nFidias P , et al. Genotypic and histological evolution of lung cancers \nacquiring resistance to EGFR inhibitors. Sci Transl Med. 2011;3:75ra26.\n 82. Oser MG, Niederst MJ, Sequist LV, Engelman JA. Transformation from \nnon-small-cell lung cancer to small-cell lung cancer: Molecular drivers \nand cells of origin. Lancet Oncol. 2015;16:e165-172.\n 83. Norkowski E, Ghigna MR, Lacroix L, Le Chevalier T, Fadel E, Dartevelle \nP , et al. Small-cell carcinoma in the setting of pulmonary adenocarci-\nnoma: new insights in the era of molecular pathology. J Thorac Oncol. \n2013;8:1265–71.\n 84. Yin X, Li Y, Wang H, Jia T, Wang E, Luo Y, et al. Small cell lung cancer \ntransformation: from pathogenesis to treatment. Semin Cancer Biol. \n2022. https:// doi. org/ 10. 1016/j. semca ncer. 2022. 03. 006.\n 85. Schoenfeld AJ, Chan JM, Kubota D, Sato H, Rizvi H, Daneshbod Y, \net al. Tumor analyses reveal squamous transformation and off-target \nalterations as early resistance mechanisms to first-line osimertinib in \nEGFR-mutant lung cancer. Clin Cancer Res. 2020;26:2654–63.\n 86. Garassino MC, Cho BC, Kim JH, Mazieres J, Vansteenkiste J, Lena H, et al. \nDurvalumab as third-line or later treatment for advanced non-small-\ncell lung cancer (ATLANTIC): an open-label, single-arm, phase 2 study. \nLancet Oncol. 2018;19:521–36.\n 87. Gainor JF, Shaw AT, Sequist LV, Fu X, Azzoli CG, Piotrowska Z, et al. EGFR \nmutations and ALK rearrangements are associated with low response \nrates to PD-1 pathway blockade in non-small cell lung cancer: a retro -\nspective analysis. Clin Cancer Res. 2016;22:4585–93.\n 88. Haratani K, Hayashi H, Tanaka T, Kaneda H, Togashi Y, Sakai K, et al. \nTumor immune microenvironment and nivolumab efficacy in EGFR \nmutation-positive non-small-cell lung cancer based on T790M status \nafter disease progression during EGFR-TKI treatment. Ann Oncol. \n2017;28:1532–9.\n 89. Zheng Y, Hao S, Xiang C, Han Y, Shang Y, Zhen Q, et al. The cor -\nrelation between SPP1 and immune escape of EGFR mutant lung adenocarcinoma was explored by bioinformatics analysis. Front Oncol. \n2021;11:592854.\n 90. Peng S, Wang R, Zhang X, Ma Y, Zhong L, Li K, et al. EGFR-TKI resistance \npromotes immune escape in lung cancer via increased PD-L1 expres-\nsion. Mol Cancer. 2019;18:165.\n 91. Yu HA, Tian SK, Drilon AE, Borsu L, Riely GJ, Arcila ME, et al. Acquired \nresistance of EGFR-mutant lung cancer to a T790M-specific EGFR inhibi-\ntor: emergence of a third mutation (C797S) in the EGFR tyrosine kinase \ndomain. JAMA Oncol. 2015;1:982–4.\n 92. Wang Z, Yang JJ, Huang J, Ye JY, Zhang XC, Tu HY, et al. Lung adeno -\ncarcinoma harboring EGFR T790M and in trans C797S responds to \ncombination therapy of first- and third-generation EGFR TKIs and shifts \nallelic configuration at resistance. J Thorac Oncol. 2017;12:1723–7.\n 93. Niederst MJ, Hu H, Mulvey HE, Lockerman EL, Garcia AR, Piotrowska \nZ, et al. The allelic context of the C797S mutation acquired upon \ntreatment with third-generation EGFR inhibitors impacts sensitivity to \nsubsequent treatment strategies. Clin Cancer Res. 2015;21:3924–33.\n 94. Tsai CJ, Nussinov R. Emerging allosteric mechanism of EGFR activation \nin physiological and pathological contexts. Biophys J. 2019;117:5–13.\n 95. Engel J, Richters A, Getlik M, Tomassi S, Keul M, Termathe M, et al. Target -\ning drug resistance in EGFR with covalent inhibitors: a structure-based \ndesign approach. J Med Chem. 2015;58:6844–63.\n 96. Jia Y, Yun CH, Park E, Ercan D, Manuia M, Juarez J, et al. Overcoming \nEGFR(T790M) and EGFR(C797S) resistance with mutant-selective allos-\nteric inhibitors. Nature. 2016;534:129–32.\n 97. Lee S, Kim J, Duggirala KB, Go A, Shin I, Cho BC, et al. Allosteric \ninhibitor TREA-0236 containing non-hydrolysable quinazoline-4-one \nfor EGFR T790M/C797S mutants inhibition. Bull Korean Chem Soc. \n2018;39:895–8.\n 98. To C, Jang J, Chen T, Park E, Mushajiang M, De Clercq DJH, et al. Single \nand dual targeting of mutant EGFR with an allosteric inhibitor. Cancer \nDiscov. 2019;9:926–43.\n 99. Maity S, Pai KSR, Nayak Y. Advances in targeting EGFR allosteric site as \nanti-NSCLC therapy to overcome the drug resistance. Pharmacol Rep. \n2020;72:799–813.\n 100. De Clercq DJH, Heppner DE, To C, Jang J, Park E, Yun C-H, et al. Discov-\nery and optimization of dibenzodiazepinones as allosteric mutant-\nselective EGFR inhibitors. ACS Med Chem Lett. 2019;10:1549–53.\n 101. Duplessis M, Goergler A, Jaeschke G, Kocer B, Kuhn B, Lazarski K, et al. \nCOMPOUNDS. Publication number: 20210079005, March 18, 2021.\n 102. Lu X, Zhang T, Zhu SJ, Xun Q, Tong L, Hu X, et al. Discovery of JND3229 \nas a new EGFR(C797S) mutant inhibitor with in vivo monodrug efficacy. \nACS Med Chem Lett. 2018;9:1123–7.\n 103. Engel J, Becker C, Lategahn J, Keul M, Ketzer J, Muhlenberg T, et al. \nInsight into the inhibition of drug-resistant mutants of the receptor \ntyrosine kinase EGFR. Angew Chem Int Ed Engl. 2016;55:10909–12.\n 104. Gunther M, Lategahn J, Juchum M, Doring E, Keul M, Engel J, et al. \nTrisubstituted pyridinylimidazoles as potent inhibitors of the clini-\ncally resistant L858R/T790M/C797S EGFR mutant: targeting of both \nhydrophobic regions and the phosphate binding site. J Med Chem. \n2017;60:5613–37.\n 105. Park H, Jung HY, Mah S, Hong S. Discovery of EGF receptor inhibi-\ntors that are selective for the D746–750/T790M/C797S mutant \nthrough structure-based de novo design. Angew Chem Int Ed Engl. \n2017;56:7634–8.\n 106. Zhang M, Wang Y, Wang J, Liu Z, Shi J, Li M, et al. Design, synthesis and \nbiological evaluation of the quinazoline derivatives as L858R/T790M/\nC797S triple mutant epidermal growth factor receptor tyrosine kinase \ninhibitors. Chem Pharm Bull (Tokyo). 2020;68:971–80.\n 107. Shen J, Zhang T, Zhu SJ, Sun M, Tong L, Lai M, et al. Structure-based \ndesign of 5-methylpyrimidopyridone derivatives as new wild-type spar -\ning inhibitors of the epidermal growth factor receptor triple mutant \n(EGFR(L858R/T790M/C797S). J Med Chem. 2019;62:7302–8.\n 108. Zhang H, Wang J, Shen Y, Wang HY, Duan WM, Zhao HY, et al. Discovery \nof 2,4,6-trisubstitued pyrido[3,4-d]pyrimidine derivatives as new EGFR-\nTKIs. Eur J Med Chem. 2018;148:221–37.\n 109. Hei YY, Shen Y, Wang J, Zhang H, Zhao HY, Xin M, et al. Synthesis and \nevaluation of 2,9-disubstituted 8-phenylthio/phenylsulfinyl-9H-purine \nas new EGFR inhibitors. Bioorg Med Chem. 2018;26:2173–85.Page 42 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n 110. Lei H, Fan S, Zhang H, Liu YJ, Hei YY, Zhang JJ, et al. Discovery of novel \n9-heterocyclyl substituted 9H-purines as L858R/T790M/C797S mutant \nEGFR tyrosine kinase inhibitors. Eur J Med Chem. 2020;186:111888.\n 111. Lategahn J, Keul M, Klovekorn P , Tumbrink HL, Niggenaber J, Muller MP , \net al. Inhibition of osimertinib-resistant epidermal growth factor recep -\ntor EGFR-T790M/C797S. Chem Sci. 2019;10:10789–801.\n 112. Hu X, Xun Q, Zhang T, Zhu S-J, Li Q, Tong L, et al. 2-Oxo-\n3,4-dihydropyrimido[4,5-d] pyrimidines as new reversible inhibi-\ntors of EGFR C797S (Cys797 to Ser797) mutant. Chin Chem Lett. \n2020;31:1281–7.\n 113. Su Z, Yang T, Wang J, Lai M, Tong L, Wumaier G, et al. Design, synthesis \nand biological evaluation of potent EGFR kinase inhibitors against 19D/\nT790M/C797S mutation. Bioorg Med Chem Lett. 2020;30:127327.\n 114. Lee Kwangho, SHIN Inji, CHOI Gildon, CHAE Chong Hak, Choe Hyeon \nJeong, JUNG Myoung Eun, et al. N2,N4-diphenylpyrimidine-2,4-diamine \nderivative, method for preparing same, and pharmaceutical composi-\ntion containing same as active ingredient for prevention or treatment \nof cancer. WO2018230934, 2018.\n 115. Wu L, Liu X, Ding CZ, Chen S, Hu L, Zhao L, et al. Spiro-aryl-phosphorus-\noxygen compound as fourth generation of EGFR kinase inhibitor. WO \n2018108064 A1, 2016.\n 116. Iwao M, Fukuda T, Ishibashi F, Uehara Y, Nishiya N, Oku Y, et al. Fourth-\ngeneration EGFR tyrosine kinase inhibitor. CN 110461850 A, 2019.\n 117. Boese D, Dahmann G, Engelhardt H, Petronczki M, Scharn D. New \nbenzimidazole compounds and derivatives as EGFR inhibitors. WO \n2019162323 A1, 2019.\n 118. Ding K, Ding J, Shen J, Geng M, Lu X, Xie H, et al. Pyrimidopyridone or \npyridopyridone compound and use thereof. WO 2019015593 A1, 2019.\n 119. Ferlenghi F, Scalvini L, Vacondio F, Castelli R, Bozza N, Marseglia G, \net al. A sulfonyl fluoride derivative inhibits EGFR(L858R/T790M/C797S) \nby covalent modification of the catalytic lysine. Eur J Med Chem. \n2021;225:113786.\n 120. Morabito A, Piccirillo MC, Falasconi F, De Feo G, Del Giudice A, Bryce \nJ, et al. Vandetanib (ZD6474), a dual inhibitor of vascular endothelial \ngrowth factor receptor (vEGFR) and epidermal growth factor receptor \n(EGFR) tyrosine kinases: Current status and future directions. Oncolo -\ngist. 2009;14:378–90.\n 121. Li Q, Zhang T, Li S, Tong L, Li J, Su Z, et al. Discovery of potent and \nnoncovalent reversible EGFR kinase inhibitors of EGFR(L858R/T790M/\nC797S). ACS Med Chem Lett. 2019;10:869–73.\n 122. Wittlinger F, Heppner DE, To C, Gunther M, Shin BH, Rana JK, et al. \nDesign of a “two-in-one” mutant-selective epidermal growth factor \nreceptor inhibitor that spans the orthosteric and allosteric sites. J Med \nChem. 2022;65:1370–83.\n 123. Yu HA, Arcila ME, Rekhtman N, Sima CS, Zakowski MF, Pao W, et al. \nAnalysis of tumor specimens at the time of acquired resistance to EGFR-\nTKI therapy in 155 patients with EGFR-mutant lung cancers. Clin Cancer \nRes. 2013;19:2240–7.\n 124. Noble ME, Endicott JA, Johnson LN. Protein kinase inhibitors: Insights \ninto drug design from structure. Science. 2004;303:1800–5.\n 125. Bondeson DP , Mares A, Smith IE, Ko E, Campos S, Miah AH, et al. Cata-\nlytic in vivo protein knockdown by small-molecule protacs. Nat Chem \nBiol. 2015;11:611–7.\n 126. Winter GE, Buckley DL, Paulk J, Roberts JM, Souza A, Dhe-Paganon S, \net al. Drug development. Phthalimide conjugation as a strategy for \nin vivo target protein degradation. Science. 2015;348:1376–81.\n 127. Scheepstra M, Hekking KFW, van Hijfte L, Folmer RHA. Bivalent ligands \nfor protein degradation in drug discovery. Comput Struct Biotechnol J. \n2019;17:160–76.\n 128. An S, Fu L. Small-molecule protacs: an emerging and promising \napproach for the development of targeted therapy drugs. EBioMedi-\ncine. 2018;36:553–62.\n 129. Chamberlain PP , Hamann LG. Development of targeted protein degra-\ndation therapeutics. Nat Chem Biol. 2019;15:937–44.\n 130. Churcher I. Protac-induced protein degradation in drug discov-\nery: breaking the rules or just making new ones? J Med Chem. \n2018;61:444–52.\n 131. Cromm PM, Crews CM. Targeted protein degradation: from chemical \nbiology to drug discovery. Cell Chem Biol. 2017;24:1181–90.\n 132. Toure M, Crews CM. Small-molecule protacs: new approaches to pro -\ntein degradation. Angew Chem Int Ed Engl. 2016;55:1966–73. 133. Jang J, To C, De Clercq DJH, Park E, Ponthier CM, Shin BH, et al. Mutant-\nselective allosteric EGFR degraders are effective against a broad range \nof drug-resistant mutations. Angew Chem Int Ed Engl. 2020;59:14481–9.\n 134. Zhao HY, Yang XY, Lei H, Xi XX, Lu SM, Zhang JJ, et al. Discovery of \npotent small molecule protacs targeting mutant EGFR. Eur J Med \nChem. 2020;208:112781.\n 135. Qu X, Liu H, Song X, Sun N, Zhong H, Qiu X, et al. Effective degradation \nof  EGFRL858R+T790M mutant proteins by CRBN-based PROTAC s through \nboth proteosome and autophagy/lysosome degradation systems. Eur J \nMed Chem. 2021;218:113328.\n 136. Zheng M, Huo J, Gu X, Wang Y, Wu C, Zhang Q, et al. Rational design \nand synthesis of novel dual PROTACS for simultaneous degradation of \nEGFR and PARP . J Med Chem. 2021;64:7839–52.\n 137. Kim JH, Nam B, Choi YJ, Kim SY, Lee JE, Sung KJ, et al. Enhanced \nglycolysis supports cell survival in EGFR-mutant lung adenocarcinoma \nby inhibiting autophagy-mediated EGFR degradation. Cancer Res. \n2018;78:4482–96.\n 138. Takahashi D, Arimoto H. Targeting selective autophagy by AUTAC \ndegraders. Autophagy. 2020;16:765–6.\n 139. Takahashi D, Moriyama J, Nakamura T, Miki E, Takahashi E, Sato A, et al. \nAutacs: Cargo-specific degraders using selective autophagy. Mol Cell. \n2019;76(797–810):e10.\n 140. Ramalingam SS, Vansteenkiste J, Planchard D, Cho BC, Gray JE, Ohe \nY, et al. Overall survival with osimertinib in untreated, EGFR-mutated \nadvanced NSCLC. N Engl J Med. 2020;382:41–50.\n 141. Chau CH, Steeg PS, Figg WD. Antibody-drug conjugates for cancer. \nLancet. 2019;394:793–804.\n 142. He K, Xu J, Liang J, Jiang J, Tang M, Ye X, et al. Discovery of a novel EGFR-\ntargeting antibody-drug conjugate, SHR-A1307, for the treatment of \nsolid tumors resistant or refractory to anti-EGFR therapies. Mol Cancer \nTher. 2019;18:1104–14.\n 143. Xu R-h, Qiu M-Z, Zhang Y, Wei X-L, Hu C. First-in-human dose-escalation \nstudy of anti-EGFR adc MRG003 in patients with relapsed/refractory \nsolid tumors. J Clin Oncol. 2020;38:3550–50.\n 144. Li Z, Wang M, Yao X, Luo W, Qu Y, Yu D, et al. Development of a novel \nEGFR-targeting antibody-drug conjugate for pancreatic cancer therapy. \nTarget Oncol. 2019;14:93–105.\n 145. Shi P , Oh YT, Zhang G, Yao W, Yue P , Li Y, et al. Met gene amplification \nand protein hyperactivation is a mechanism of resistance to both first \nand third generation EGFR inhibitors in lung cancer treatment. Cancer \nLett. 2016;380:494–504.\n 146. Giroux-Leprieur E, Dumenil C, Chinet T. Combination of crizotinib and \nosimertinib or erlotinib might overcome MET-mediated resistance to \nEGFR tyrosine kinase inhibitor in EGFR-mutated adenocarcinoma. J \nThorac Oncol. 2018;13:e232–4.\n 147. Kang J, Chen HJ, Wang Z, Liu J, Li B, Zhang T, et al. Osimertinib and \ncabozantinib combinatorial therapy in an EGFR-mutant lung adeno -\ncarcinoma patient with multiple MET secondary-site mutations after \nresistance to crizotinib. J Thorac Oncol. 2018;13:e49–53.\n 148. Fujino T, Suda K, Mitsudomi T. Emerging MET tyrosine kinase inhibitors \nfor the treatment of non-small cell lung cancer. Expert Opin Emerg \nDrugs. 2020;25:229–49.\n 149. Quintanal-Villalonga A, Molina-Pinelo S, Cirauqui C, Ojeda-Marquez L, \nMarrugal A, Suarez R, et al. FGFR1 cooperates with EGFR in lung cancer \noncogenesis, and their combined inhibition shows improved efficacy. J \nThorac Oncol. 2019;14:641–55.\n 150. Shaw AT, Felip E, Bauer TM, Besse B, Navarro A, Postel-Vinay S, et al. Lor -\nlatinib in non-small-cell lung cancer with ALK or ROS1 rearrangement: \nan international, multicentre, open-label, single-arm first-in-man phase \n1 trial. Lancet Oncol. 2017;18:1590–9.\n 151. Uchibori K, Inase N, Araki M, Kamada M, Sato S, Okuno Y, et al. Brigatinib \ncombined with anti-EGFR antibody overcomes osimertinib resist -\nance in EGFR-mutated non-small-cell lung cancer. Nat Commun. \n2017;8:14768.\n 152. Liu S, Li S, Hai J, Wang X, Chen T, Quinn MM, et al. Targeting HER2 aber -\nrations in non-small cell lung cancer with osimertinib. Clin Cancer Res. \n2018;24:2594–604.\n 153. La Monica S, Cretella D, Bonelli M, Fumarola C, Cavazzoni A, Digiacomo \nG, et al. Trastuzumab emtansine delays and overcomes resistance to \nthe third-generation EGFR-TKI osimertinib in NSCLC EGFR mutated cell \nlines. J Exp Clin Cancer Res. 2017;36:174.Page 43 of 44\n Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n \n 154. Jani JP , Arcari J, Bernardo V, Bhattacharya SK, Briere D, Cohen BD, et al. \nPF-03814735, an orally bioavailable small molecule aurora kinase inhibi-\ntor for cancer therapy. Mol Cancer Ther. 2010;9:883–94.\n 155. Kim C, Giaccone G. MEK inhibitors under development for treatment of \nnon-small-cell lung cancer. Expert Opin Investig Drugs. 2018;27:17–30.\n 156. Ortiz-Cuaran S, Scheffler M, Plenker D, Dahmen L, Scheel AH, \nFernandez-Cuesta L, et al. Heterogeneous mechanisms of primary and \nacquired resistance to third-generation EGFR inhibitors. Clin Cancer Res. \n2016;22:4837–47.\n 157. Della Corte CM, Ciaramella V, Cardone C, La Monica S, Alfieri R, Petronini \nPG, et al. Antitumor efficacy of dual blockade of EGFR signaling by \nosimertinib in combination with selumetinib or cetuximab in activated \nEGFR human NCLC tumor models. J Thorac Oncol. 2018;13:810–20.\n 158. Jacobsen K, Bertran-Alamillo J, Molina MA, Teixido C, Karachaliou N, \nPedersen MH, et al. Convergent Akt activation drives acquired EGFR \ninhibitor resistance in lung cancer. Nat Commun. 2017;8:410.\n 159. Namba K, Shien K, Takahashi Y, Torigoe H, Sato H, Yoshioka T, et al. Acti-\nvation of AXL as a preclinical acquired resistance mechanism against \nosimertinib treatment in EGFR-mutant non-small cell lung cancer cells. \nMol Cancer Res. 2019;17:499–507.\n 160. Jimbo T, Hatanaka M, Komatsu T, Taira T, Kumazawa K, Maeda N, et al. \nDS-1205b, a novel selective inhibitor of AXL kinase, blocks resistance \nto EGFR-tyrosine kinase inhibitors in a non-small cell lung cancer xeno -\ngraft model. Oncotarget. 2019;10:5152–67.\n 161. Kim D, Bach DH, Fan YH, Luu TT, Hong JY, Park HJ, et al. AXL degrada-\ntion in combination with EGFR-TKI can delay and overcome acquired \nresistance in human non-small cell lung cancer cells. Cell Death Dis. \n2019;10:361.\n 162. Liu YN, Tsai MF, Wu SG, Chang TH, Tsai TH, Gow CH, et al. Acquired \nresistance to EGFR tyrosine kinase inhibitors is mediated by the \nreactivation of STC2/JUN/AXL signaling in lung cancer. Int J Cancer. \n2019;145:1609–24.\n 163. Gu J, Qian L, Zhang G, Mahajan NP , Owonikoko TK, Ramalingam SS, \net al. Inhibition of ACK1 delays and overcomes acquired resistance \nof EGFR mutant NSCLC cells to the third generation EGFR inhibitor, \nosimertinib. Lung Cancer. 2020;150:26–35.\n 164. Lawrence HR, Mahajan K, Luo Y, Zhang D, Tindall N, Huseyin M, et al. \nDevelopment of novel ACK1/TNK2 inhibitors using a fragment-based \napproach. J Med Chem. 2015;58:2746–63.\n 165. Sequist LV, Lynch TJ. EGFR tyrosine kinase inhibitors in lung cancer: an \nevolving story. Annu Rev Med. 2008;59:429–42.\n 166. Kummar S, Chen HX, Wright J, Holbeck S, Millin MD, Tomaszewski J, \net al. Utilizing targeted cancer therapeutic agents in combination: \nnovel approaches and urgent requirements. Nat Rev Drug Discov. \n2010;9:843–56.\n 167. Anighoro A, Bajorath J, Rastelli G. Polypharmacology: challenges and \nopportunities in drug discovery. J Med Chem. 2014;57:7874–87.\n 168. Chen G, Bao Y, Weng Q, Zhao Y, Lu X, Fu L, et al. Compound 15c, a novel \ndual inhibitor of EGFR(L858R/T790M) and FGFR1, efficiently overcomes \nepidermal growth factor receptor-tyrosine kinase inhibitor resistance of \nnon-small-cell lung cancers. Front Pharmacol. 2019;10:1533.\n 169. Cui Z, Chen S, Wang Y, Gao C, Chen Y, Tan C, et al. Design, synthesis and \nevaluation of azaacridine derivatives as dual-target EGFR and Src kinase \ninhibitors for antitumor treatment. Eur J Med Chem. 2017;136:372–81.\n 170. Mansour TS, Pallepati RR, Basetti V. Potent dual EGFR/HER4 tyrosine \nkinase inhibitors containing novel (1,2-dithiolan-4-yl)acetamides. \nBioorg Med Chem Lett. 2020;30:127288.\n 171. El-Sayed NA, Nour MS, Salem MA, Arafa RK. New oxadiazoles with selec-\ntive- COX-2 and EGFR dual inhibitory activity: design, synthesis, cytotox -\nicity evaluation and in silico studies. Eur J Med Chem. 2019;183:111693.\n 172. Abdelatef SA, El-Saadi MT, Amin NH, Abdelazeem AH, Omar HA, \nAbdellatif KRA. Design, synthesis and anticancer evaluation of novel \nspirobenzo[h]chromene and spirochromane derivatives with dual EGFR \nand B-RAF inhibitory activities. Eur J Med Chem. 2018;150:567–78.\n 173. Jang J, Son JB, To C, Bahcall M, Kim SY, Kang SY, et al. Discovery of \na potent dual ALK and EGFR T790M inhibitor. Eur J Med Chem. \n2017;136:497–510.\n 174. Chen Y, Wu J, Wang A, Qi Z, Jiang T, Chen C, et al. Discovery of n-(5-((5-\nchloro-4-((2-(isopropylsulfonyl)phenyl)amino)pyrimidin-2-yl)amino)-4-\nmet hoxy-2-(4-methyl-1,4-diazepan-1-yl)phenyl)acrylamide (chmfl-alk/\nEGFR-050) as a potent ALK/EGFR dual kinase inhibitor capable of overcoming a variety of ALK/EGFR associated drug resistant mutants in \nNSCLC. Eur J Med Chem. 2017;139:674–97.\n 175. Jing T, Miao X, Jiang F, Guo M, Xing L, Zhang J, et al. Discovery and \noptimization of tetrahydropyrido[4,3-d]pyrimidine derivatives as novel \nATX and EGFR dual inhibitors. Bioorg Med Chem. 2018;26:1784–96.\n 176. Kurup S, McAllister B, Liskova P , Mistry T, Fanizza A, Stanford D, et al. \nDesign, synthesis and biological activity of n(4)-phenylsubstituted-\n7h-pyrrolo[2,3-d]pyrimidin-4-amines as dual inhibitors of aurora kinase \na and epidermal growth factor receptor kinase. J Enzyme Inhib Med \nChem. 2018;33:74–84.\n 177. Gadekar PK, Urunkar G, Roychowdhury A, Sharma R, Bose J, \nKhanna S, et al. Design, synthesis and biological evaluation of \n2,3-dihydroimidazo[2,1-b]thiazoles as dual EGFR and IGF1R inhibitors. \nBioorg Chem. 2021;115:105151.\n 178. Romagnoli R, Prencipe F, Oliva P , Baraldi S, Baraldi PG, Schiaffino Ortega \nS, et al. Design, synthesis, and biological evaluation of 6-substi-\ntuted thieno[3,2-d]pyrimidine analogues as dual epidermal growth \nfactor receptor kinase and microtubule inhibitors. J Med Chem. \n2019;62:1274–90.\n 179. Alswah M, Bayoumi AH, Elgamal K, Elmorsy A, Ihmaid S, Ahmed HEA. \nDesign, synthesis and cytotoxic evaluation of novel chalcone deriva-\ntives bearing triazolo[4,3-a]-quinoxaline moieties as potent anticancer \nagents with dual EGFR kinase and tubulin polymerization inhibitory \neffects. Molecules. 2017;23:48.\n 180. Khan I, Garikapati KR, Setti A, Shaik AB, Kanth Makani VK, Shareef MA, \net al. Design, synthesis, in silico pharmacokinetics prediction and \nbiological evaluation of 1,4-dihydroindeno[1,2-c]pyrazole chalcone as \nEGFR/AKT pathway inhibitors. Eur J Med Chem. 2019;163:636–48.\n 181. Dong H, Yin H, Zhao C, Cao J, Xu W, Zhang Y. Design, synthesis and \nbiological evaluation of novel osimertinib-based HDAC and EGFR dual \ninhibitors. Molecules. 2019;24:2407.\n 182. Fischer T, Najjar A, Totzke F, Schachtele C, Sippl W, Ritter C, et al. Dis-\ncovery of novel dual inhibitors of receptor tyrosine kinases EGFR and \nPDGFR-β related to anticancer drug resistance. J Enzyme Inhib Med \nChem. 2018;33:1–8.\n 183. Hamed MM, Darwish SS, Herrmann J, Abadi AH, Engel M. First bispecific \ninhibitors of the epidermal growth factor receptor kinase and the NF-κB \nactivity as novel anticancer agents. J Med Chem. 2017;60:2853–68.\n 184. Dokla EME, Fang CS, Abouzid KAM, Chen CS. 1,2,4-oxadiazole deriva-\ntives targeting EGFR and c-Met degradation in TKI resistant NSCLC. Eur J \nMed Chem. 2019;182:111607.\n 185. Singh PK, Silakari O. Molecular dynamics guided development of \nindole based dual inhibitors of EGFR (T790M) and c-Met. Bioorg Chem. \n2018;79:163–70.\n 186. Fischer T, Kruger T, Najjar A, Totzke F, Schachtele C, Sippl W, et al. Discov-\nery of novel substituted benzo-anellated 4-benzylamino pyrrolopyri-\nmidines as dual EGFR and vEGFR2 inhibitors. Bioorg Med Chem Lett. \n2017;27:2708–12.\n 187. Zhang HQ, Gong FH, Ye JQ, Zhang C, Yue XH, Li CG, et al. Design and \ndiscovery of 4-anilinoquinazoline-urea derivatives as dual TK inhibitors \nof EGFR and vEGFR-2. Eur J Med Chem. 2017;125:245–54.\n 188. Wei H, Duan Y, Gou W, Cui J, Ning H, Li D, et al. Design, synthesis and \nbiological evaluation of novel 4-anilinoquinazoline derivatives as \nhypoxia-selective EGFR and vEGFR-2 dual inhibitors. Eur J Med Chem. \n2019;181:111552.\n 189. Sun S, Zhang J, Wang N, Kong X, Fu F, Wang H, et al. Design and discov-\nery of quinazoline- and thiourea-containing sorafenib analogs as EGFR \nand vEGFR-2 dual TK inhibitors. Molecules. 2017;23:24.\n 190. Das D, Xie L, Wang J, Xu X, Zhang Z, Shi J, et al. Discovery of new quina-\nzoline derivatives as irreversible dual EGFR/HER2 inhibitors and their \nanticancer activities: part 1. Bioorg Med Chem Lett. 2019;29:591–6.\n 191. Maher M, Kassab AE, Zaher AF, Mahmoud Z. Novel pyrazolo[3,4-d]\npyrimidines: design, synthesis, anticancer activity, dual EGFR/ErbB2 \nreceptor tyrosine kinases inhibitory activity, effects on cell cycle profile \nand caspase-3-mediated apoptosis. J Enzyme Inhib Med Chem. \n2019;34:532–46.\n 192. Zou M, Li J, Jin B, Wang M, Chen H, Zhang Z, et al. Design, synthesis \nand anticancer evaluation of new 4-anilinoquinoline-3-carbonitrile \nderivatives as dual EGFR/HER2 inhibitors and apoptosis inducers. Bioorg \nChem. 2021;114:105200.Page 44 of 44 Shi\xa0et\xa0al. Journal of Hematology & Oncology           (2022) 15:94 \n•\n fast, convenient online submission\n •\n  thorough peer review by experienced researchers in your ﬁeld\n• \n rapid publication on acceptance\n• \n support for research data, including large and complex data types\n•\n  gold Open Access which fosters wider collaboration and increased citations \n maximum visibility for your research: over 100M website views per year •\n  At BMC, research is always in progress.\nLearn more biomedcentral.com/submissionsReady to submit y our researc h Ready to submit y our researc h  ?  Choose BMC and benefit fr om: ?  Choose BMC and benefit fr om: \n 193. Alsaid MS, Al-Mishari AA, Soliman AM, Ragab FA, Ghorab MM. Discovery \nof benzo[g]quinazolin benzenesulfonamide derivatives as dual EGFR/\nHER2 inhibitors. Eur J Med Chem. 2017;141:84–91.\n 194. Ghorab MM, Alsaid MS, Soliman AM. Dual EGFR/HER2 inhibitors and \napoptosis inducers: new benzo[g]quinazoline derivatives bearing \nbenzenesulfonamide as anticancer and radiosensitizers. Bioorg Chem. \n2018;80:611–20.\n 195. Soliman AM, Alqahtani AS, Ghorab M. Novel sulphonamide benzo -\nquinazolinones as dual EGFR/HER2 inhibitors, apoptosis inducers and \nradiosensitizers. J Enzyme Inhib Med Chem. 2019;34:1030–40.\n 196. Liu X, Du Q, Tian C, Tang M, Jiang Y, Wang Y, et al. Discovery of cape \nderivatives as dual EGFR and CSK inhibitors with anticancer activ-\nity in a murine model of hepatocellular carcinoma. Bioorg Chem. \n2021;107:104536.\n 197. Zhang B, Liu Z, Xia S, Liu Q, Gou S. Design, synthesis and biological \nevaluation of sulfamoylphenyl-quinazoline derivatives as potential \nEGFR/CAIX dual inhibitors. Eur J Med Chem. 2021;216:113300.\n 198. Zang H, Qian G, Arbiser J, Owonikoko TK, Ramalingam SS, Fan S, et al. \nOvercoming acquired resistance of EGFR-mutant NSCLC cells to the \nthird generation EGFR inhibitor, osimertinib, with the natural product \nhonokiol. Mol Oncol. 2020;14:882–95.\n 199. Cao F, Gong YB, Kang XH, Lu ZH, Wang Y, Zhao KL, et al. Degradation of \nMCL-1 by bufalin reverses acquired resistance to osimertinib in EGFR-\nmutant lung cancer. Toxicol Appl Pharmacol. 2019;379:114662.\n 200. Sun P , Qu Y, Wang Y, Wang J, Wang X, Sheng J. Wighteone exhibits an \nantitumor effect against EGFR L858R/T790M mutation non-small cell \nlung cancer. J Cancer. 2021;12:3900–8.\n 201. Niu M, Xu J, Liu Y, Li Y, He T, Ding L, et al. FBXL 2 counteracts Grp94 to \ndestabilize EGFR and inhibit EGFR-driven NSCLC growth. Nat Commun. \n2021;12:5919.\n 202. Zhang KR, Zhang YF, Lei HM, Tang YB, Ma CS, Lv QM, et al. Targeting \nAKR1B1 inhibits glutathione de novo synthesis to overcome acquired \nresistance to EGFR-targeted therapy in lung cancer. Sci Transl Med. \n2021;13:eabg6428.\n 203. Hitosugi T, Zhou L, Elf S, Fan J, Kang HB, Seo JH, et al. Phosphoglycerate \nmutase 1 coordinates glycolysis and biosynthesis to promote tumor \ngrowth. Cancer Cell. 2012;22:585–600.\n 204. Liang Q, Gu WM, Huang K, Luo MY, Zou JH, Zhuang GL, et al. HKB99, an \nallosteric inhibitor of phosphoglycerate mutase 1, suppresses invasive \npseudopodia formation and upregulates plasminogen activator \ninhibitor-2 in erlotinib-resistant non-small cell lung cancer cells. Acta \nPharmacol Sin. 2021;42:115–9.\n 205. Huang K, Liang Q, Zhou Y, Jiang LL, Gu WM, Luo MY, et al. A novel allos-\nteric inhibitor of phosphoglycerate mutase 1 suppresses growth and \nmetastasis of non-small-cell lung cancer. Cell Metab. 2021;33:223.\n 206. Huang K, Liang Q, Zhou Y, Jiang L-l, Gu W-m, Luo M-y, et al. A novel \nallosteric inhibitor of phosphoglycerate mutase 1 suppresses growth \nand metastasis of non-small-cell lung cancer. Cell Metab. 2019;30:1107-\n19.e8.\n 207. Qiu Y, Yin X, Li X, Wang Y, Fu Q, Huang R, et al. Untangling dual-\ntargeting therapeutic mechanism of epidermal growth factor receptor \n(EGFR) based on reversed allosteric communication. Pharmaceutics. \n2021;13:747.\n 208. Yin L, Zhang Y, Yin L, Ou Y, Lewis MS, Wang R, et al. Novel mitochondria-\nbased targeting restores responsiveness in therapeutically resistant \nhuman lung cancer cells. Mol Cancer Ther. 2021;20(12):2527–38.\n 209. He J, Huang Z, Han L, Gong Y, Xie C. Mechanisms and management of \n3rd-generation EGFR-TKI resistance in advanced non-small cell lung \ncancer (Review). Int J Oncol. 2021;59:90.\n 210. Planchard D, Feng PH, Karaseva N, Kim SW, Kim TM, Lee CK, et al. \nOsimertinib plus platinum–pemetrexed in newly diagnosed epidermal \ngrowth factor receptor mutation-positive advanced/metastatic non-\nsmall-cell lung cancer: safety run-in results from the FLAURA2 study. \nESMO Open. 2020;6:100271.\n 211. Yi M, Zheng X, Niu M, Zhu S, Ge H, Wu K. Combination strategies with \nPD-1/PD-L1 blockade: current advances and future directions. Mol \nCancer. 2022;21:28.\n 212. Gandhi L, Rodriguez-Abreu D, Gadgeel S, Esteban E, Felip E, De Angelis \nF, et al. Pembrolizumab plus chemotherapy in metastatic non-small-cell \nlung cancer. N Engl J Med. 2018;378:2078–92. 213. Paz-Ares L, Luft A, Vicente D, Tafreshi A, Gumus M, Mazieres J, et al. \nPembrolizumab plus chemotherapy for squamous non-small-cell lung \ncancer. N Engl J Med. 2018;379:2040–51.\n 214. Zhou C, Wu L, Fan Y, Wang Z, Liu L, Chen G, et al. Sintilimab plus plati-\nnum and gemcitabine as first-line treatment for advanced or metastatic \nsquamous nsclc: results from a randomized, double-blind, phase 3 trial \n(ORIENT-12). J Thorac Oncol. 2021;16:1501–11.\n 215. Socinski MA, Jotte RM, Cappuzzo F, Orlandi F, Stroyakovskiy D, Nogami \nN, et al. Atezolizumab for first-line treatment of metastatic nonsqua-\nmous NSCLC. N Engl J Med. 2018;378:2288–301.\n 216. Jabbour SK, Berman AT, Decker RH, Lin Y, Feigenberg SJ, Gettinger SN, \net al. Phase 1 trial of pembrolizumab administered concurrently with \nchemoradiotherapy for locally advanced non-small cell lung cancer: a \nnonrandomized controlled trial. JAMA Oncol. 2020;6:848–55.\n 217. Liu D, Gong J, Liu T, Li K, Yin X, Liu Y, et al. Phase 1 study of SHR-1701, a \nbifunctional fusion protein targeting PD-L1 and TGF-β, in patients with \nadvanced solid tumors. J Clin Oncol. 2021;39:2503–2503.\n 218. Sequist LV, Waltman BA, Dias-Santagata D, Digumarthy S, Turke AB, \nFidias P , et al. Genotypic and histological evolution of lung cancers \nacquiring resistance to EGFR inhibitors. Sci Trans Med. 2011;3:75ra26.\n 219. Wu L, Ke L, Zhang Z, Yu J, Meng X. Development of EGFR TKIs and \noptions to manage resistance of third-generation EGFR TKI osimertinib: \nconventional ways and immune checkpoint inhibitors. Front Oncol. \n2020;10:602762–862.\nPublisher’s Note\nSpringer Nature remains neutral with regard to jurisdictional claims in pub -\nlished maps and institutional affiliations.', 'metadata': {'source': 'Emerging strategies to overcome resistance to third-generation EGFR inhibitors.pdf'}}

In [13]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
# prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
# Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
# However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
# strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.


# QUESTION: {query_oneline}
# PASSAGE: {passage_oneline}
# """

prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
You specialise in targetted therapy for lung cancer and you are aiding in figuring our how to overcome resistance using known information given. 
You may look to open sources to suggest methods relevent to the questions. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.


QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
You specialise in targetted therapy for lung cancer and you are aiding in figuring our how to overcome resistance using known information given. 
You may look to open sources to suggest methods relevent to the questions. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.


QUESTION: The EGFR T790 has a mutation that causes steric hindrance, where is the attachement on the EGFR protein does this happen, what is the amino acid chain of this mutation, what can be done to clip of steric hindrance?
PASSAGE: {'content': 'Shi\xa0 et\xa0al. Journal of Hematology & Oncology           (2022) 15:94  \nhttps://doi.or

In [14]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

The EGFR T790M mutation, a substitution of threonine with methionine at position 790 within the EGFR protein's tyrosine kinase domain, creates steric hindrance.  This occurs in the ATP binding site of the EGFR protein, specifically impacting the interaction of the drug with the hydrophobic region near the gatekeeper residue. The amino acid chain directly involved is the single amino acid change from threonine (Thr) to methionine (Met) at position 790.  Overcoming this steric hindrance can be achieved through several approaches, including the development of allosteric inhibitors which bind to a different site on the protein, causing a conformational change that disrupts the binding of the ATP competitive inhibitor, or the use of PROTAC technology to degrade the mutated EGFR protein entirely.


In [15]:
# Key Concepts in the Code
# Embeddings:

# Numerical vectors representing textual data. The embeddings for documents and queries help identify relevance by computing similarity.
# ChromaDB:

# A vector database for storing and querying documents using embeddings.
# Generative AI:

# The genai library is used to produce natural language responses, tailored to the input query and reference passage.
# Retry Mechanism:

# Ensures resilience in embedding generation when encountering transient errors.
# Conversational Prompting:

# Guides the generative model to provide answers in a specific tone and level of detail, tailored for the intended audience.

In [16]:
# 1. What the GeminiEmbeddingFunction Does
# Yes, the GeminiEmbeddingFunction converts the PDFs into embeddings. Here's how it fits into the pipeline:

# PDF Text Extraction: Text is extracted from each PDF.
# Embedding Conversion: The GeminiEmbeddingFunction processes the extracted text and converts it into embeddings using the specified model (genai.embed_content).
# 2. ChromaDB and Query Matching
# Your description of ChromaDB's role is accurate:

# Storing Embeddings: The embeddings for the extracted text are stored in ChromaDB.
# Query Handling:
# The query is also converted into an embedding by the same GeminiEmbeddingFunction (in query mode).
# ChromaDB uses cosine similarity (or a similar metric) to compare the query embedding with the stored embeddings.
# It retrieves the most relevant document(s) based on similarity scores.
# 3. Role of the Generative Model
# The generative model (genai.GenerativeModel) is used for the final step of answering the query:

# Once a relevant document is retrieved, the generative model uses the document's content and the query to construct a contextual response.
# This involves word prediction and probability scoring based on the language model, ensuring the answer is complete and conversational.
# 4. Corrected Summary
# Here’s a more precise version of the process:

# Convert PDFs to Embeddings:

# Extract text from the PDFs.
# Use GeminiEmbeddingFunction to convert the text into embeddings.
# Store Embeddings in ChromaDB:

# Add the generated embeddings to ChromaDB for future retrieval.
# Handle Query:

# Convert the input query into an embedding using the same GeminiEmbeddingFunction.
# Use ChromaDB to find the most similar document embedding via cosine similarity.
# Generate a Contextual Answer:

# Retrieve the relevant document(s) from ChromaDB.
# Use the generative model to analyze the document and query.
# Generate a natural-language response that answers the query based on the retrieved content.